In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import time,os, math
import pyarrow as pa
import pyarrow.parquet as pq
from numba import cuda
from multiprocessing import Process,Queue
from tqdm import tqdm

from tensorflow_graphics.geometry.transformation.quaternion import from_rotation_matrix
from tensorflow_graphics.geometry.transformation.rotation_matrix_3d import from_quaternion

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 텐서플로가 첫 번째 GPU에 4GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=100)])
    tf.config.experimental.set_virtual_device_configuration(
        gpus[1],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:   
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

2 Physical GPUs, 2 Logical GPUs


In [2]:
from sympy import symbols, cos, sin, pi, simplify, pprint, tan, expand_trig, sqrt, trigsimp, atan2
from sympy.matrices import Matrix
import mpmath
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import time,math
import numpy as np
import os
try:
    with tf.device('/device:GPU:1'):

        os.chdir('/media/irlab/F042305F42302CAA/6_dof_data')

        def pose(theta, d, alpha, l):
            # given the parameters:
            # theta: theta[i]
            # alpha: alpha[i]
            # a: a[i]
            # d: d[i]

            r11, r12 = cos(theta), -sin(theta)*cos(alpha)
            r23, r33 = -cos(theta)*sin(alpha), cos(alpha)
            r21 = sin(theta) 
            r22 = cos(theta) * cos(alpha)
            r13 = sin(theta) * sin(alpha)
            r32 = sin(alpha)
            y = l * sin(theta)
            z = d 
            x = l*cos(theta)

            T = Matrix([
            [r11, r12, r13, x],
            [r21, r22, r23, y],
            [0.0, r32, r33, z],
            [0.0, 0.0, 0.0, 1]
          ])

            T = simplify(T)

            return T

        # 우리가 사용할 변수들 모두 정의 :
        #DH parms 변수들 선언
        q1, q2, q3, q4, q5, q6 = symbols('q1:7')
        d1 = symbols('d1')  # d5 
        a3 = symbols('a3')  # a3 
        a2= symbols('a2')   # a2 
        d5 = symbols('d5')  # d5 
        d4 = symbols('d4')  # d4 
        d6= symbols('d6')   # a2 
        # T06 에 사용될 우리가 인풋 값을 넣어줄 변수들 ! 
        px = symbols('px')
        py = symbols('py')
        pz = symbols('pz')
        alpha= symbols('alpha')  # alpha : roll 
        beta= symbols('beta')    # beta : pitch 
        gamma= symbols('gamma')  # gamma : Yaw
        #pose(theta,d, alpha, a)
        d90 = pi/2
        T01 = pose(q1,d1,d90,0)
        T12 = pose(q2,0,0,a2)
        T23 = pose(q3,0,0,a3)
        T34 = pose(q4,d4,d90,0)
        T45 = pose(q5,d5,-d90,0)
        T56 = pose(q6,0,0,0)
        T6g = pose(0,0,0,d5)

        T02 = simplify(T01*T12)
        T03 = simplify(T02*T23)
        T04 = simplify(T03*T34)
        T05 = simplify(T04*T45)
        T06 = simplify(T05*T56)


        R0u = Matrix([
            [1.0*cos(alpha)*cos(beta), -1.0*sin(alpha)*cos(gamma) + sin(beta)*sin(gamma)*cos(alpha), 1.0*sin(alpha)*sin(gamma) + sin(beta)*cos(alpha)*cos(gamma)],
            [1.0*sin(alpha)*cos(beta),  sin(alpha)*sin(beta)*sin(gamma) + 1.0*cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - 1.0*sin(gamma)*cos(alpha)],
            [          -1.0*sin(beta),                                     1.0*sin(gamma)*cos(beta),                                    1.0*cos(beta)*cos(gamma)]])


        Tend =Matrix([
        [-sin(alpha)*sin(gamma)+cos(alpha)*cos(beta)*cos(gamma),  -cos(alpha)*sin(beta), sin(alpha)*cos(gamma)+cos(alpha)*cos(beta)*sin(gamma), px],
        [sin(beta)*cos(gamma), cos(beta) , sin(beta)*sin(gamma), py],
        [-cos(alpha)*sin(gamma)-sin(alpha)*cos(beta)*cos(gamma), sin(alpha)*sin(beta),cos(alpha)*cos(gamma)-sin(alpha)*sin(gamma)*cos(beta) , pz],
        [                       0,                                         0,                                      0,  1]])

except RuntimeError as e:
    pass

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov  1 11:59:22 2021

@author: wngks

UR3 Robot Kimenatics & Inv Kinematics
"""
#!/usr/bin/python2
## UR5/UR10 Inverse Kinematics - Ryan Keating Johns Hopkins University

# ***** lib
import numpy as np
from numpy import linalg
from random import uniform

import cmath
import math
from math import cos as cos
from math import sin as sin
from math import atan2 as atan2
from math import acos as acos
from math import asin as asin
from math import sqrt as sqrt
from math import pi as pi
from numpy.linalg import norm
global mat
from sklearn.metrics import mean_squared_error
mat=np.matrix


# ****** Coefficients ******


global d1, a2, a3, a7, d4, d5, d6
# 단위 : cm
try:
    with tf.device('/device:GPU:1'):
        d1 =  15.19   
        a2 = -24.365
        a3 = -21.3258
        d4 =  11.235
        d5 =  8.535
        d6 =  8.19
        # a7 = 0.075

        global d, a, alph
        d = mat([15.19, 0,0 ,11.235,8.535,8.19]) #ur3 cm
        a =mat([0 ,-24.365 ,-21.325 ,0 ,0 ,0]) #ur3 cm
        alph = mat([pi/2, 0, 0, pi/2, -pi/2, 0 ]) # ur3 rad

        def RMSE(a,b):
            return mean_squared_error(a,b)**0.5

        # ************************************************** FORWARD KINEMATICS
        def AH( n,th,c  ):

            T_a = mat(np.identity(4), copy=False)
            T_a[0,3] = a[0,n-1]
            T_d = mat(np.identity(4), copy=False)
            T_d[2,3] = d[0,n-1]

            Rzt = mat([[cos(th[n-1,c]), -sin(th[n-1,c]), 0 ,0],
                    [sin(th[n-1,c]),  cos(th[n-1,c]), 0, 0],
                    [0,               0,              1, 0],
                    [0,               0,              0, 1]],copy=False)


            Rxa = mat([[1, 0,                 0,                  0],
                     [0, cos(alph[0,n-1]), -sin(alph[0,n-1]),   0],
                     [0, sin(alph[0,n-1]),  cos(alph[0,n-1]),   0],
                     [0, 0,                 0,                  1]],copy=False)

            A_i = T_d * Rzt * T_a * Rxa
            return A_i

        def HTrans(th,c ):  
          A_1=AH( 1,th,c  )
          A_2=AH( 2,th,c  )
          A_3=AH( 3,th,c  )
          A_4=AH( 4,th,c  )
          A_5=AH( 5,th,c  )
          A_6=AH( 6,th,c  )

          T_06=A_1*A_2*A_3*A_4*A_5*A_6

          return T_06

        # th 위의 함수 param 설명 : 
            # th에는 array 형태의 th가 들어가고 그때, 1행은 th1, 2행 th2, ... 6행 th6; 열으로 구분 x 열은 th 집합의 개수.
            # c는 몇변째 열의 th 집합을 불러 오는지 index 값, 

        # ************************************************** INVERSE KINEMATICS 
        def invKine(desired_pos):# T60
          th = mat(np.zeros((6, 8)))
          conf = mat(np.zeros((3,8)))
          P_05 = (desired_pos * mat([0,0, -d6, 1]).T-mat([0,0,0,1 ]).T)

          # **** theta1 ****

          psi = atan2(P_05[2-1,0], P_05[1-1,0])
          phi = acos(d4 /sqrt(P_05[2-1,0]*P_05[2-1,0] + P_05[1-1,0]*P_05[1-1,0]))
          #The two solutions for theta1 correspond to the shoulder
          #being either left or right
          th[0, 0:4] = pi/2 + psi + phi
          th[0, 4:8] = pi/2 + psi - phi
          th = th.real

          # **** theta5 ****

          cl = [0, 4]# wrist up or down
          for i in range(0,len(cl)):
                  c = cl[i]
                  T_10 = linalg.inv(AH(1,th,c))
                  T_16 = T_10 * desired_pos
                  th[4, c:c+2] = + acos((T_16[2,3]-d4)/d6);
                  th[4, c+2:c+4] = - acos((T_16[2,3]-d4)/d6);

          th = th.real

          # **** theta6 ****
          # theta6 is not well-defined when sin(theta5) = 0 or when T16(1,3), T16(2,3) = 0.

          cl = [0, 2, 4, 6]
          for i in range(0,len(cl)):
                  c = cl[i]
                  T_10 = linalg.inv(AH(1,th,c))
                  T_16 = linalg.inv( T_10 * desired_pos )
                  th[5, c:c+2] = atan2((-T_16[1,2]/sin(th[4, c])),(T_16[0,2]/sin(th[4, c])))

          th = th.real

          # **** theta3 ****
          cl = [0, 2, 4, 6]
          for i in range(0,len(cl)):
                  c = cl[i]
                  T_10 = linalg.inv(AH(1,th,c))
                  T_65 = AH( 6,th,c)
                  T_54 = AH( 5,th,c)
                  T_14 = ( T_10 * desired_pos) * linalg.inv(T_54 * T_65)
                  P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0,0,0,1]).T
                  t3 = cmath.acos((linalg.norm(P_13)**2 - a2**2 - a3**2 )/(2 * a2 * a3)) # norm ?
                  th[2, c] = t3.real
                  th[2, c+1] = -t3.real

          # **** theta2 and theta 4 ****

          cl = [0, 1, 2, 3, 4, 5, 6, 7]
          for i in range(0,len(cl)):
                  c = cl[i]
                  T_10 = linalg.inv(AH( 1,th,c ))
                  T_65 = linalg.inv(AH( 6,th,c))
                  T_54 = linalg.inv(AH( 5,th,c))
                  T_14 = (T_10 * desired_pos) * T_65 * T_54
                  P_13 = T_14 * mat([0, -d4, 0, 1]).T - mat([0,0,0,1]).T

                  # theta 2
                  th[1, c] = -atan2(P_13[1], -P_13[0]) + asin(a3* sin(th[2,c])/linalg.norm(P_13))
                  # theta 4
                  T_32 = linalg.inv(AH( 3,th,c))
                  T_21 = linalg.inv(AH( 2,th,c))
                  T_34 = T_32 * T_21 * T_14
                  th[3, c] = atan2(T_34[1,0], T_34[0,0])
          th = th.real

          return th

        def get_invK(pos):
            inv_pos = np.zeros

        def cos_sim(A, B):   #유사도 검사하는 함수
               return np.dot(A, B)/(norm(A)*norm(B))

        def get_pos(p):
            # p : position 1x6 array
            a= round(p[3],5)
            b=round(p[4],5)
            g=round(p[5],5)
            x_=round(p[0],5)
            y_=round(p[1],5)
            z_=round(p[2],5)
            return mat([[cos(a)*cos(b)*cos(g) - sin(a)*sin(g) ,-cos(a)*sin(b),cos(g)*sin(a) + cos(a)*cos(b)*sin(g),x_],
                        [cos(g)*sin(b),                          cos(b),        sin(b)*sin(g),                       y_],
                        [-cos(a)*sin(g) - cos(b)*cos(g)*sin(a),sin(a)*sin(b), cos(a)*cos(g) - cos(b)*sin(a)*sin(g), z_],
                        [0,0,0,1]])

        def prediction(n,conf_=0):
            x_ = copy.deepcopy(x_tst[n])

            if conf_<=0 or conf_>8:
                print("데이터 그대로")
            else:
                x_[-1] = conf_*10
                print("conf 변경")

            x_ = np.expand_dims(x_,axis=0)

            pred = trained_model.predict(x_)
            x_[0][3:6] /= 10
            #print(x_)

            print('input position : ',x_)
            print(conf_)

            print('answer theta : ',y_tst[n])

            pred_= np.array([pred]).T

            end_point = HTrans(pred_,0)
            #pos_conf[3:6] *= 10

            x1 = end_point[0,3]
            y1 = end_point[1,3]
            z1 = end_point[2,3]
            a = (atan2(end_point[2,1],-end_point[0,1]))
            b = (atan2(sqrt(1-end_point[1,1]**2),end_point[1,1] ))
            g = (atan2(end_point[1,2],end_point[1,0]))
            print('pred theta : ',pred)
            print()
            new_pos = np.array([x1,y1,z1,a,b,g]).astype('float32')

            in_pos= get_pos(x_[0])
            recon_pos = get_pos(new_pos)
            print(in_pos,"input_pos ")
            print()
            print(recon_pos,"reconstruct_pos")
            #print(x_[:6])
            #print(np.array([x1,y1,z1,a,b,g]).astype('float32'))



            #print( 'similarity : ',cos_sim(in_pos[0],recon_pos[0])) # 코사인 유사도 평가
            print('RMSE :',RMSE(np.asarray(in_pos[0]),np.asarray(recon_pos[0]))) # 평균 제곱근 오차

        def prediction_2(n,conf_=0):
            x_ = copy.deepcopy(x_tst[n])

            if conf_<=0 or conf_>8:
                print("데이터 그대로")
            else:
                x_[-1] = conf_*10
                print("conf 변경")

            x_ = np.expand_dims(x_,axis=0)
            x_[0][3:6] /= 10

            pred = trained_model.predict(x_)
            #print(x_)

            print('input position : ',x_)
            print(conf_)

            print('answer theta : ',y_tst[n])

            pred_= np.array([pred]).T

            end_point = HTrans(pred_,0)
            #pos_conf[3:6] *= 10

            x1 = end_point[0,3]
            y1 = end_point[1,3]
            z1 = end_point[2,3]
            a = (atan2(end_point[2,1],-end_point[0,1]))
            b = (atan2(sqrt(1-end_point[1,1]**2),end_point[1,1] ))
            g = (atan2(end_point[1,2],end_point[1,0]))
            print('pred theta : ',pred)
            print()
            new_pos = np.array([x1,y1,z1,a,b,g]).astype('float32')

            in_pos= get_pos(x_[0])
            recon_pos = get_pos(new_pos)
            print(in_pos,"input_pos ")
            print()
            print(recon_pos,"reconstruct_pos")

        def prediction2(n,conf_=0):
            x_ = copy.deepcopy(x_tst2[n])

            if conf_<=0 or conf_>8:
                print("데이터 그대로")
            else:
                x_[-1] = conf_*10
                print("conf 변경")

            x_ = np.expand_dims(x_,axis=0)

            pred = trained_model.predict(x_)
            x_[0][3:6] /= 10
            #print(x_)

            print('input position : ',x_)
            print(conf_)

            print('answer theta : ',y_tst2[n])

            pred_= np.array([pred]).T

            end_point = HTrans(pred_,0)
            #pos_conf[3:6] *= 10

            x1 = end_point[0,3]
            y1 = end_point[1,3]
            z1 = end_point[2,3]
            a = (atan2(end_point[2,1],-end_point[0,1]))
            b = (atan2(sqrt(1-end_point[1,1]**2),end_point[1,1] ))
            g = (atan2(end_point[1,2],end_point[1,0]))
            print('pred theta : ',pred)
            print()
            new_pos = np.array([x1,y1,z1,a,b,g]).astype('float32')

            in_pos= get_pos(x_[0])
            recon_pos = get_pos(new_pos)
            print(in_pos,"input_pos ")
            print(recon_pos,"reconstruct_pos")
            #print(x_[:6])
            #print(np.array([x1,y1,z1,a,b,g]).astype('float32'))


            #print( 'similarity : ',cos_sim(in_pos[0],recon_pos[0])) # 코사인 유사도 평가
            print('RMSE :',RMSE(np.asarray(in_pos[0]),np.asarray(recon_pos[0]))) # 평균 제곱근 오차
            
except RuntimeError as e:
    pass

In [4]:
try:
    with tf.device('/device:GPU:1'):

        class UR3_DNN:
            def __init__(self,bs):
                self.batch_size = bs
                self.d1 =  15.19   
                self.a2 = -24.365
                self.a3 = -21.3258
                self.d4 =  11.235
                self.d5 =  8.535
                self.d6 =  8.19
                self.alpha1=pi/2
                self.alpha2=pi/2
                self.alpha3=-pi/2

                self.d = mat([self.d1, 0,0 ,self.d4,self.d5,self.d6]) #ur3 m
                self.a =mat([0 ,self.a2 ,self.a3 ,0 ,0 ,0]) #ur3 cm
                self.alpha = mat([self.alpha1, 0, 0, self.alpha2, self.alpha3, 0 ]) # ur3 rad

                self.position_data = np.array([[0,0,0,0,0,0,0]])  #  input data :  x,y,z,a,b,g,conf
                self.th_data = np.array([[0,0,0,0,0,0]])          #  output data : th1~th6
                self.th_sample = np.array([[0,0,0,0,0,0]])          #  output data : th1~th6
                self.position_sample= np.array([[0.,0.,0.,0.,0.,0.]], dtype='float32')

                self.identity_flattened = tf.repeat(tf.reshape(tf.eye(4), (1, 16)), self.batch_size, 0) 

                self.zeros = tf.repeat(tf.reshape(0., (1)), self.batch_size, 0) 
                self.ones = tf.repeat(tf.reshape(1., (1)), self.batch_size, 0) 
                self.pi_2 = tf.repeat(tf.reshape(pi/2, (1)), self.batch_size, 0) 

                self.d1_batch = tf.repeat([self.d1], self.batch_size,0)
                self.a2_batch = tf.repeat([self.a2], self.batch_size,0)
                self.a3_batch = tf.repeat([self.a3], self.batch_size,0)
                self.d4_batch = tf.repeat([self.d4], self.batch_size,0)
                self.d5_batch = tf.repeat([self.d5], self.batch_size,0)
                self.d6_batch = tf.repeat([self.d6], self.batch_size,0)

                self.d_ = [self.d1_batch, self.zeros,self.zeros ,self.d4_batch,self.d5_batch,self.d6_batch] #ur3 m
                self.a_ =[self.zeros ,self.a2_batch ,self.a3_batch ,self.zeros ,self.zeros ,self.zeros] #ur3 cm
                self.alpha_ = [self.pi_2, self.zeros, self.zeros, self.pi_2, -self.pi_2, self.zeros] # ur3 rad

            def RMSE(self,a,b):
                return mean_squared_error(a,b)**0.5

            # ***************************** FORWARD KINEMATICS
            def AH(self, n,th,c  ):

                T_a = mat(np.identity(4), copy=False)
                T_a[0,3] = self.a[0,n-1]
                T_d = mat(np.identity(4), copy=False)
                T_d[2,3] = self.d[0,n-1]

                Rzt = mat([[cos(th[n-1,c]), -sin(th[n-1,c]), 0 ,0],
                        [sin(th[n-1,c]),  cos(th[n-1,c]), 0, 0],
                        [0,               0,              1, 0],
                        [0,               0,              0, 1]],copy=False)


                Rxa = mat([[1, 0,                 0,                  0],
                         [0, cos(self.alpha[0,n-1]), -sin(self.alpha[0,n-1]),   0],
                         [0, sin(self.alpha[0,n-1]),  cos(self.alpha[0,n-1]),   0],
                         [0, 0,                 0,                  1]],copy=False)

                A_i = T_d * Rzt * T_a * Rxa
                return A_i


            def HTrans(self,th,c ):  
                A_1=self.AH( 1,th,c  )
                A_2=self.AH( 2,th,c  )
                A_3=self.AH( 3,th,c  )
                A_4=self.AH( 4,th,c  )
                A_5=self.AH( 5,th,c  )
                A_6=self.AH( 6,th,c  )

                T_06=A_1*A_2*A_3*A_4*A_5*A_6

                return T_06
            #####################################################################################
            #tensor로 들어가도록 구현, error 구하고 array size가 [:,6] 형태로 들어가게함

            #tf 가능한 정기구학 gkat함수구현
            def AH_tf(self,n,th):   

                s_t = tf.sin(th[:,n-1])
                c_t = tf.cos(th[:,n-1])

                s_a = tf.sin(self.alpha_[n-1])
                c_a = tf.cos(self.alpha_[n-1])

                s_t = tf.cast(s_t,tf.float32)
                c_t = tf.cast(c_t,tf.float32)

                #A_i = [c_t, -c_a*s_t, s_a*s_t,l*c_t,
                #      s_t, c_a*c_t, -s_a*c_t, l*s_t,
                #      0, s_a, c_a, d
                #      0, 0, 0, 1]
                A_ = [c_t, -c_a*s_t, s_a*s_t, self.a_[n-1]*c_t,
                      s_t, c_a*c_t, -s_a*c_t, self.a_[n-1]*s_t,
                      self.zeros, s_a, c_a, self.d_[n-1],
                      self.zeros,self.zeros,self.zeros,self.ones]
                A_ = tf.reshape(tf.stack(A_, axis=1), (self.batch_size, 4, 4))
                return A_

            def HTrans_tf(self,th ):  
                self.identity_flattened = tf.repeat(tf.reshape(tf.eye(4), (1, 16)), self.batch_size, 0) 

                self.zeros = tf.repeat(tf.reshape(0., (1)), self.batch_size, 0) 
                self.ones = tf.repeat(tf.reshape(1., (1)), self.batch_size, 0) 
                self.pi_2 = tf.repeat(tf.reshape(pi/2, (1)), self.batch_size, 0) 

                self.d1_batch = tf.repeat([self.d1], self.batch_size,0)
                self.a2_batch = tf.repeat([self.a2], self.batch_size,0)
                self.a3_batch = tf.repeat([self.a3], self.batch_size,0)
                self.d4_batch = tf.repeat([self.d4], self.batch_size,0)
                self.d5_batch = tf.repeat([self.d5], self.batch_size,0)
                self.d6_batch = tf.repeat([self.d6], self.batch_size,0)

                self.d_ = [self.d1_batch, self.zeros,self.zeros ,self.d4_batch,self.d5_batch,self.d6_batch] #ur3 m
                self.a_ =[self.zeros ,self.a2_batch ,self.a3_batch ,self.zeros ,self.zeros ,self.zeros] #ur3 cm
                self.alpha_ = [self.pi_2, self.zeros, self.zeros, self.pi_2, -self.pi_2, self.zeros] # ur3 rad

                #th= tf.convert_to_tensor(th)
                #th= tf.transpose(th)
                A_1=self.AH_tf( 1,th )
                A_2=self.AH_tf( 2,th )
                A_3=self.AH_tf( 3,th )
                A_4=self.AH_tf( 4,th )
                A_5=self.AH_tf( 5,th )
                A_6=self.AH_tf( 6,th )

                A_ = tf.matmul(A_1,A_2)
                A_ = tf.matmul(A_,A_3)
                A_ = tf.matmul(A_,A_4)
                A_ = tf.matmul(A_,A_5)
                A_ = tf.matmul(A_,A_6)

                return A_
            # ************************************************** INVERSE KINEMATICS 
            def invKine(self,desired_pos):# T60
                th = mat(np.zeros((6, 8)))
                conf = mat(np.zeros((3,8)))
                P_05 = (desired_pos * mat([0,0, -self.d6, 1]).T-mat([0,0,0,1 ]).T)

                # **** theta1 ****

                psi = atan2(P_05[2-1,0], P_05[1-1,0])

                phi = acos(self.d4 /sqrt(P_05[2-1,0]*P_05[2-1,0] + P_05[1-1,0]*P_05[1-1,0]))
                #The two solutions for theta1 correspond to the shoulder
                #being either left or right
                th[0, 0:4] = pi/2 + psi + phi
                th[0, 4:8] = pi/2 + psi - phi
                th = th.real

                # **** theta5 ****

                cl = [0, 4]# wrist up or down
                for i in range(0,len(cl)):
                    c = cl[i]
                    T_10 = linalg.inv(self.AH(1,th,c))
                    T_16 = T_10 * desired_pos

                    th[4, c:c+2] = acos((T_16[2,3]-self.d4)/self.d6)
                    th[4, c+2:c+4] = -acos((T_16[2,3]-self.d4)/self.d6)


                th = th.real

                # **** theta6 ****
                # theta6 is not well-defined when sin(theta5) = 0 or when T16(1,3), T16(2,3) = 0.

                cl = [0, 2, 4, 6]
                for i in range(0,len(cl)):
                    c = cl[i]
                    T_10 = linalg.inv(self.AH(1,th,c))
                    T_16 = linalg.inv( T_10 * desired_pos )
                    th[5, c:c+2] = atan2((-T_16[1,2]/sin(th[4, c])),(T_16[0,2]/sin(th[4, c])))

                th = th.real

                # **** theta3 ****
                cl = [0, 2, 4, 6]
                for i in range(0,len(cl)):
                    c = cl[i]
                    T_10 = linalg.inv(self.AH(1,th,c))
                    T_65 = self.AH( 6,th,c)
                    T_54 = self.AH( 5,th,c)
                    T_14 = ( T_10 * desired_pos) * linalg.inv(T_54 * T_65)
                    P_13 = T_14 * mat([0, -self.d4, 0, 1]).T - mat([0,0,0,1]).T
                    t3 = cmath.acos((linalg.norm(P_13)**2 - self.a2**2 - self.a3**2 )/(2 * self.a2 * self.a3)) # norm ?
                    th[2, c] = t3.real
                    th[2, c+1] = -t3.real

                # **** theta2 and theta 4 ****

                cl = [0, 1, 2, 3, 4, 5, 6, 7]
                for i in range(0,len(cl)):
                    c = cl[i]
                    T_10 = linalg.inv(self.AH( 1,th,c ))
                    T_65 = linalg.inv(self.AH( 6,th,c))
                    T_54 = linalg.inv(self.AH( 5,th,c))
                    T_14 = (T_10 * desired_pos) * T_65 * T_54
                    P_13 = T_14 * mat([0, -self.d4, 0, 1]).T - mat([0,0,0,1]).T

                    # theta 2
                    th[1, c] = -atan2(P_13[1], -P_13[0]) + asin(self.a3* sin(th[2,c])/linalg.norm(P_13))
                    # theta 4
                    T_32 = linalg.inv(self.AH( 3,th,c))
                    T_21 = linalg.inv(self.AH( 2,th,c))
                    T_34 = T_32 * T_21 * T_14
                    th[3, c] = atan2(T_34[1,0], T_34[0,0])
                th = th.real
                return th

            def cos_sim(self,A, B):   # Check for similarity
                return np.dot(A, B)/(norm(A)*norm(B))

            def get_pos(self,p):
                # p : position 1x6 array
                a= round(p[3],5)
                b=round(p[4],5)
                g=round(p[5],5)
                x_=round(p[0],5)
                y_=round(p[1],5)
                z_=round(p[2],5)
                return mat([[cos(a)*cos(b)*cos(g) - sin(a)*sin(g) ,-cos(a)*sin(b),cos(g)*sin(a) + cos(a)*cos(b)*sin(g),x_],
                            [cos(g)*sin(b),                          cos(b),        sin(b)*sin(g),                       y_],
                            [-cos(a)*sin(g) - cos(b)*cos(g)*sin(a),sin(a)*sin(b), cos(a)*cos(g) - cos(b)*sin(a)*sin(g), z_],
                            [0,0,0,1]])

            def get_position(self,p):
                # p : N x 4 x 4  tensor
                x = p[:,0,3]
                y = p[:,1,3]
                z = p[:,2,3]

                cos = p[:,1,1]

                sin = tf.sqrt(1-tf.math.square(cos))
                b = tf.where(p[:,1,1]>=0, tf.math.atan2(sin,cos), tf.math.atan2(- sin,cos))
                a = tf.atan2(p[:,2,1],-p[:,0,1])
                g = tf.atan2(p[:,1,2],p[:,1,0])

                pos_ = np.array([x,y,z,a,b,g])
                pos_= pos_.T
                pos_ = tf.convert_to_tensor(
                    pos_, dtype=None, dtype_hint=None, name=None
                )

            def data_sample(self,n):
                inv_th1 = 0
                data_cnt = 0
                pass_cnt = 0 
                t = time.time()
                position_sample = np.array([[0,0,0,0,0,0,0]])  #  input data :  x,y,z,a,b,g,conf
                th_sample = np.array([[0,0,0,0,0,0]])          #  output data : th1~th6

                with tqdm(total= n) as pbar:
                    while (data_cnt<=n):
                        x = uniform(-50.0,50.0)
                        y = uniform(-50.0,50.0)    
                        z = uniform(-50.0,50.0)
                        alpha_ = uniform(-math.pi,math.pi)
                        beta_ = uniform(-math.pi,math.pi)
                        gamma_ = uniform(-math.pi,math.pi)
                        r = sqrt(x**2+y**2+z**2)
                        if r<50. and r>20.:
                            pos = [x,y,z,alpha_,beta_,gamma_]
                            pos_1=self.get_pos(pos)
                            try:
                                inv_th1 = self.invKine(pos_1)
                            except:
                                pass
                            inv_pos1_2 = self.HTrans(inv_th1,0) 
                            error = round(self.RMSE(pos_1.tolist(),inv_pos1_2.tolist()),6)
                            if (2e-04 >=error):
                                data_cnt += 1

                                is_there_minus = inv_th1[0,:] < 0 
                                is_there_plus = inv_th1[0,:] > 0 

                                # if th1 is all negative; Check for amniotic fluid If yes ? pass, correct if not
                                if not(True in is_there_plus):
                                    min_th1 = min(inv_th1[0,0],inv_th1[0,4])
                                    if inv_th1[0,0]==min_th1:
                                        inv_th1[0,:4] += 2*math.pi 
                                    else:
                                        inv_th1[0,4:] += 2*math.pi 

                                # if th1 is all positive; Check for negative numbers If yes ? pass, correct if not
                                if not(True in is_there_minus):
                                    max_th1 = max(inv_th1[0,0],inv_th1[0,4])
                                    if inv_th1[0,0]==max_th1:
                                        inv_th1[0,:4] -= 2*math.pi 
                                    else:
                                        inv_th1[0,4:] -= 2*math.pi 

        #                         pos_conf = np.hstack([pos,0])
                                #pos_conf[3:6]*=10
                                for i in range(8):
        #                             conf = 0
                                    th_tem = inv_th1[:,i].T
                                    th_tem = th_tem.tolist()[0]
        #                             if th_tem[2] == 0:
        #                                 continue
        #                             if (th_tem[0] >= 0 and th_tem[2] >= 0 and th_tem[4] >= 0):
        #                                 conf = 80
        #                             elif (th_tem[0] >= 0 and th_tem[2] >= 0 and th_tem[4] < 0):
        #                                 conf = 70
        #                             elif (th_tem[0] >= 0 and th_tem[2] < 0 and th_tem[4] >= 0):
        #                                 conf = 60
        #                             elif (th_tem[0] >= 0 and th_tem[2] < 0 and th_tem[4] < 0):
        #                                 conf = 50
        #                             elif (th_tem[0] < 0 and th_tem[2] >= 0 and th_tem[4] >= 0):
        #                                 conf = 40
        #                             elif (th_tem[0] < 0 and th_tem[2] >= 0 and th_tem[4] < 0):
        #                                 conf = 30
        #                             elif (th_tem[0] < 0 and th_tem[2] < 0 and th_tem[4] >= 0):
        #                                 conf = 20
        #                             elif (th_tem[0] < 0 and th_tem[2] < 0 and th_tem[4] < 0):
        #                                 conf = 10
        #                             pos_conf[6] = conf
        #                             position_sample = np.vstack([position_sample,pos_conf])
                                    th_sample = np.vstack([th_sample,th_tem])
                                pass_cnt += 1
                                pbar.update(1)
                            else:
                                pass
                        else:        
                            continue
        #         self.position_data = np.vstack([self.position_data,position_sample[1:]])
                self.th_data = np.vstack([self.th_data,th_sample[1:]])
                for i in range(2):
                    self.th_data[:,i] = np.where(self.th_data[:,i]>pi, self.th_data[:,i]-2*pi , self.th_data[:,i])
                    self.th_data[:,i] = np.where(self.th_data[:,i]<-pi, self.th_data[:,i]+2*pi , self.th_data[:,i])

                #self.th_data = np.where(self.th_data<0, self.th_data+2*pi , self.th_data)
                #self.position_data[:,3:6] = np.where(self.position_data[:,3:6]<0,self.position_data[:,3:6] + 2*pi, self.position_data[:,3:6])
                print("time : ",time.time()-t)
                print("pass : ",pass_cnt)
                print("data : ",data_cnt)

            def data_sample_ths(self,n):
                inv_th1 = 0
                data_cnt = 0
                pass_cnt = 0 
                t = time.time()
                sample = np.array([[0.,0.,0.,0.,0.,0.]], dtype='float32')
                with tqdm(total= n) as pbar:
                    while(data_cnt < n):
                        th1 = uniform(-math.pi,math.pi)
                        th2 = uniform(-math.pi,math.pi)
                        th3 = uniform(-math.pi,math.pi)
                        th4 = uniform(-math.pi,math.pi)
                        th5 = uniform(-math.pi,math.pi)
                        th6 = uniform(-math.pi,math.pi)
                        th = [th1,th2,th3,th4,th5,th6]

                        sample = np.vstack([sample,th])
                        pbar.update(1)
                        data_cnt += 1
                self.th_sample = np.vstack([sample[1:],self.th_sample[1:]])




                print("time : ",time.time()-t)
                print("pass : ",pass_cnt)
                print("data : ",data_cnt)


            def get_pos_data(self,mat,ths):
                # mat : N x 4 x 4  Tensor Pos
                # pos : N x 8    x,y,z a,b,g,w, conf
                pos_ = mat[:,:3,3]

                quat_ = from_rotation_matrix(mat[:,:3,:3])

                conf= mat[:,0,0]

                print(pos_.shape)
                conf = conf.numpy()
                ths = ths.numpy()
                conf[np.where((ths[:,0]>=0) & (ths[:,2]>=0) & (ths[:,4]>=0))] = 80
                conf[np.where((ths[:,0]>=0) & (ths[:,2]>=0) & (ths[:,4]<0))] = 70
                conf[np.where((ths[:,0]>=0) & (ths[:,2]<0) & (ths[:,4]>=0))] = 60
                conf[np.where((ths[:,0]>=0) & (ths[:,2]<0) & (ths[:,4]<0))] = 50
                conf[np.where((ths[:,0]<0) & (ths[:,2]>=0) & (ths[:,4]>=0))] = 40
                conf[np.where((ths[:,0]<0) & (ths[:,2]>=0) & (ths[:,4]<0))] = 30
                conf[np.where((ths[:,0]<0) & (ths[:,2]<0) & (ths[:,4]>=0))] = 20
                conf[np.where((ths[:,0]<0) & (ths[:,2]<0) & (ths[:,4]<0))] = 10

                conf= tf.convert_to_tensor([conf])
                conf = tf.transpose(conf)

                return tf.experimental.numpy.hstack([pos_,quat_,conf])




            def get_position(self,p,output_):
                # N x 4 x 4   Tensor pos
                # N x 6  Tensor ths
                x = p[:,0,3]
                y = p[:,1,3]
                z = p[:,2,3]

                #cos = p[:,1,1]

                #sin = tf.sqrt(1-tf.math.square(cos))
                b = tf.acos(p[:,1,1])
                a = tf.atan2(p[:,2,1]/tf.math.sin(b),-p[:,0,1]/tf.math.sin(b))
                g = tf.atan2(p[:,1,2]/tf.math.sin(b),p[:,1,0]/tf.math.sin(b))

                conf= x

                conf = conf.numpy()

                output = output_.numpy()

                conf[np.where((output[:,0]>=0) & (output[:,2]>=0) & (output[:,4]>=0))] = 80
                conf[np.where((output[:,0]>=0) & (output[:,2]>=0) & (output[:,4]<0))] = 70
                conf[np.where((output[:,0]>=0) & (output[:,2]<0) & (output[:,4]>=0))] = 60
                conf[np.where((output[:,0]>=0) & (output[:,2]<0) & (output[:,4]<0))] = 50
                conf[np.where((output[:,0]<0) & (output[:,2]>=0) & (output[:,4]>=0))] = 40
                conf[np.where((output[:,0]<0) & (output[:,2]>=0) & (output[:,4]<0))] = 30
                conf[np.where((output[:,0]<0) & (output[:,2]<0) & (output[:,4]>=0))] = 20
                conf[np.where((output[:,0]<0) & (output[:,2]<0) & (output[:,4]<0))] = 10

                pos_ = np.array([x,y,z,a,b,g,conf])
                pos_= pos_.T
                pos_ = tf.convert_to_tensor(
                    pos_, dtype=None, dtype_hint=None, name=None
                )


                return pos_

            def data_split(self,data_y,data):
                x_train, x_test, y_train, y_test = train_test_split(data_y, data, test_size=0.2, shuffle=True)#데이터셋 전처리
                x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.5, shuffle=True)#데이터셋 전처리
                return x_train,y_train,x_test,y_test,x_valid,y_valid


            def filter_data(self,ths,pos):

                r = tf.math.sqrt(tf.square(robot.position_data[:,0])+tf.square(robot.position_data[:,1])+tf.square(robot.position_data[:,2]))
                r = r.numpy()

                ths = ths.numpy()
                pos = pos.numpy()



                pos = pos[np.where((r<55) & (r>5) )]
                ths = ths[np.where((r<55) & (r>5) )]

                #ths = ths[np.where(pos[:,2]>0)]
                #pos = pos[np.where(pos[:,2]>0)]

                #ths = np.where(ths<0, ths+2*pi , ths)

                #for i in range(3,6):
                #    pos[:,i] = np.where(pos[:,i]<0,pos[:,i] + 2*pi, pos[:,i])

                ths= tf.convert_to_tensor(ths)
                pos= tf.convert_to_tensor(pos)


                return ths,pos

except RuntimeError as e:
    pass


In [5]:
def yzy_euler_to_quat(p):
    # p = position_data : x,y,z,a,b,g,conf
    
    p = tf.convert_to_tensor(
    p, dtype=None, dtype_hint=None, name=None
    )
    
    
    ths = p[:,3:6]
        
    half_angles = ths / 2.0
    cos_half_angles = tf.cos(half_angles)
    sin_half_angles = tf.sin(half_angles)
    
    c1, c2, c3 = tf.unstack(cos_half_angles, axis=-1)
    s1, s2, s3 = tf.unstack(sin_half_angles, axis=-1)
    
    w = c1*c2*c3 - s1*c2*s3
    x = s1*s2*c3 - c1*s2*s3
    y = c1*c2*s3 +s1*c2*c3
    z = c1*s2*c3 + s1*s2*s3
    
    return tf.stack((p[:,0], p[:,1] ,p[:,2] ,x, y, z, w, p[:,-1]), axis=-1)


def quat_to_yzy_euler(p):
    
    p = tf.convert_to_tensor(
    p, dtype=None, dtype_hint=None, name=None
    )
    
    c_b = tf.sqrt(tf.square(p[:,6]) + tf.square(p[:,4]))
    s_b = tf.sqrt(tf.square(p[:,5]) + tf.square(p[:,3]))
    
    s = tf.atan2(p[:,4],p[:,6])
    d = tf.atan2(p[:,3],p[:,5])
    
    a = s+d
    g = s-d
    
    
    b = tf.where(c_b != 0, 2*tf.atan2(s_b,c_b), 0 )

    return tf.stack((p[:,0], p[:,1] ,p[:,2] ,a,b,g, p[:,-1]), axis=-1)

In [180]:
from random import *
import time

try:
    with tf.device('/device:GPU:1'):

        d1 =  15.19   
        a2 = -24.365
        a3 = -21.3258
        d4 =  11.235
        d5 =  8.535
        d6 =  8.19
        alpha1=pi/2
        alpha2=pi/2
        alpha3=-pi/2


        #alph = mat([pi/2, 0, 0, pi/2, -pi/2, 0 ]) # ur3 rad

        position_data = np.array([[0,0,0,0,0,0,0]])  ##   for x,y,z,a,b,g,conf

        #for 'UR3_Dense_robot_config.h5' 모델 용
        #position_data = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])   ##   for x,y,z,a,b,g,conf,d1,a2,a3,d4,d5,d6,a1,a4,a5

        #th_data = np.array([[0,0,0,0,0,0,0,0,0]])
        th_data = np.array([[0,0,0,0,0,0]])
        data_cnt = 0
        pass_cnt = 0 
        inv_th1 = 0
        t = time.time()
        while(data_cnt<=200):
            x = uniform(-50.00,50.00)
            y = uniform(-50.00,50.00)    
            z = uniform(-50.00,50.00)

            alpha_ = uniform(-math.pi,math.pi)
            beta_ = uniform(-math.pi,math.pi)
            gamma_ = uniform(-math.pi,math.pi)
            
            r = sqrt(x**2+y**2+z**2)

            if r<55 and r>5:
                pos = [x,y,z,alpha_,beta_,gamma_]

                pos_1=get_pos(pos)
                try:
                    inv_th1 = invKine(pos_1)
                except:
                    pass
                inv_pos1_2 = HTrans(inv_th1,0) 

                if (2e-04 >= round(RMSE(pos_1.tolist(),inv_pos1_2.tolist()),6)):
                    data_cnt += 1

                    is_there_minus = inv_th1[0,:] < 0 
                    is_there_plus = inv_th1[0,:] > 0 

                    # th1이 모두 음수일 경우; 양수가 있는지 검사 만약 있으면 ? 통과, 없으면 수정 
                    if not(True in is_there_plus):
                        min_th1 = min(inv_th1[0,0],inv_th1[0,4])
                        if inv_th1[0,0]==min_th1:
                            inv_th1[0,:4] += 2*math.pi 
                        else:
                            inv_th1[0,4:] += 2*math.pi 

                    # th1이 모두 양수일 경우; 음수가 있는지 검사 만약 있으면 ? 통과, 없으면 수정 
                    if not(True in is_there_minus):
                        max_th1 = max(inv_th1[0,0],inv_th1[0,4])
                        if inv_th1[0,0]==max_th1:
                            inv_th1[0,:4] -= 2*math.pi 
                        else:
                            inv_th1[0,4:] -= 2*math.pi 

                    #inv_th1 = np.where(inv_th1<-pi, inv_th1+2*pi, inv_th1)
                    #inv_th1 = np.where(inv_th1>pi, inv_th1-2*pi, inv_th1)    

                    #pos_conf = np.hstack([pos,0,0,0]) # one-hot version
                    #pos_conf = np.hstack([pos,0,d1,a2,a3,d4,d5,d6,alpha1,alpha2,alpha3])
                    pos_conf = np.hstack([pos,0])
                    #pos_conf[3:6]*=10
                    
                    #inv_th1[0:4:] *=10  # radian 값에 10배해서 -31,4~31,4
                    #inv_th1 *=10
                    for i in range(8):

                        #conf = [0,0,0]  
                        conf = 0
                        th_tem = inv_th1[:,i].T
                        th_tem = th_tem.tolist()[0]
                        #print(th_tem)
                        if th_tem[2] == 0:
                            #print("Th3 is 0,  pass")
                            continue

                        if (th_tem[0] >= 0 and th_tem[2] >= 0 and th_tem[4] >= 0):
                            #conf = [5,5,5]
                            conf = 80
                        elif (th_tem[0] >= 0 and th_tem[2] >= 0 and th_tem[4] < 0):
                            #conf = [5,5,0]
                            conf = 70
                        elif (th_tem[0] >= 0 and th_tem[2] < 0 and th_tem[4] >= 0):
                            #conf = [5,0,5]
                            conf = 60
                        elif (th_tem[0] >= 0 and th_tem[2] < 0 and th_tem[4] < 0):
                            #conf = [5,0,0]
                            conf = 50
                        elif (th_tem[0] < 0 and th_tem[2] >= 0 and th_tem[4] >= 0):
                            #conf = [0,5,5]
                            conf = 40
                        elif (th_tem[0] < 0 and th_tem[2] >= 0 and th_tem[4] < 0):
                            #conf = [0,5,0]
                            conf = 30
                        elif (th_tem[0] < 0 and th_tem[2] < 0 and th_tem[4] >= 0):
                            #conf = [0,0,5]
                            conf = 20
                        elif (th_tem[0] < 0 and th_tem[2] < 0 and th_tem[4] < 0):
                            #conf = [0,0,0]
                            conf = 10

                        #pos_conf[-1] = conf[2]
                        #pos_conf[-2] = conf[1]
                        #pos_conf[-3] = conf[0]
                        pos_conf[6] = conf
                        position_data = np.vstack([position_data,pos_conf])
                        th_data = np.vstack([th_data,th_tem])
                else:
                    pass
                #print( )        
            else:        
                pass_cnt += 1
                #print("data_num : ",data_cnt)
                continue
            th_data[:,0] = np.where(th_data[:,0]>pi, th_data[:,0]-2*pi , th_data[:,0])
            th_data[:,0] = np.where(th_data[:,0]<-pi, th_data[:,0]+2*pi , th_data[:,0])
            th_data[:,1] = np.where(th_data[:,1]>pi, th_data[:,1]-2*pi , th_data[:,1])
            th_data[:,1] = np.where(th_data[:,1]<-pi, th_data[:,1]+2*pi , th_data[:,1])
        print("time : ",time.time()-t)
        print("pass : ",pass_cnt)
        print("data : ",data_cnt)
except RuntimeError as e:
    pass


time :  2.6391990184783936
pass :  146
data :  201


In [213]:
print(robot.th_data[10])
print(robot.position_data[10])

[ 2.0968475  6.8682237 11.342129   8.519645  18.487593   4.4643364]
[ 11.981999 -14.142469  13.891232 -21.277607  17.38457  -21.06251
  50.      ]


In [276]:
robot.data_sample(40000)

40001it [13:25, 49.67it/s]                           

time :  805.412032365799
pass :  40001
data :  40001


In [267]:
robot = UR3_DNN(500)

In [19]:
print(np.where(th_data[:,0]>pi))
print(len(th_data))
print(len(position_data))

position_data[:,3:6] /= 10

(array([], dtype=int64),)
35017
35017


In [274]:
## 뽑고싶은 개수만큼 뽑아 

n = 20000
robot.batch_size= n
robot.data_sample_ths(n)

robot.batch_size= 500

100%|██████████| 20000/20000 [00:00<00:00, 23963.56it/s]

time :  0.8384461402893066
pass :  0
data :  20000


In [178]:
euler_[10]

<tf.Tensor: shape=(7,), dtype=float64, numpy=
array([ 8.59708868, 25.47789242,  2.08675782, -0.76117823,  0.8551389 ,
        2.86840563, 60.        ])>

In [179]:
euler_2[10]

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([ 8.597088  , 25.477888  ,  2.0867596 , -0.76118267,  0.85514003,
        2.868403  , 60.        ], dtype=float32)>

In [72]:
robot.th_data = robot.th_data[:round(len(robot.th_data),-3)]


In [222]:
len(robot.th_data)

420004

In [195]:
n=5
print(th_data[n:n+8])
print(position_data[n:n+8])

tf.Tensor(
[[ 2.19056531 -2.96505867  1.77841208  1.58284867  0.91838483  1.00523596]
 [ 2.19056531 -1.35025719 -1.77841208 -2.75831397  0.91838483  1.00523596]
 [ 2.19056531  2.58194268  2.0396452  -1.08379315 -0.91838483 -2.1363567 ]
 [ 2.19056531 -1.87726707 -2.0396452   1.17152169 -0.91838483 -2.1363567 ]
 [ 2.91408779 -0.82742767  1.43300829 -2.06811163  1.45401216  1.81711623]
 [ 2.91408779  0.48985098 -1.43300829 -0.51937371  1.45401216  1.81711623]
 [ 0.28928373  2.66524559  1.42556708 -2.67164706 -1.61909305  1.30299346]
 [ 0.28928373 -2.30723758 -1.42556708 -1.13121504 -1.61909305  1.30299346]], shape=(8, 6), dtype=float64)
[[ 5.47686002 20.22727205 28.85511058  1.09897377  2.80216581 -0.82290684
  80.        ]
 [ 5.47686002 20.22727205 28.85511058  1.09897377  2.80216581 -0.82290684
  60.        ]
 [ 5.47686002 20.22727205 28.85511058  1.09897377  2.80216581 -0.82290684
  70.        ]
 [ 5.47686002 20.22727205 28.85511058  1.09897377  2.80216581 -0.82290684
  50.        ]
 [

In [168]:
try:
    with tf.device('/device:GPU:1'):
        quat_ = yzy_euler_to_quat(position_data)
        quat_2 = yzy_euler_to_quat(position_)

        euler_ =quat_to_yzy_euler(quat_)
        euler_2 =quat_to_yzy_euler(quat_2)
except RuntimeError as e:
    pass


tf.Tensor(
[ 0.          2.41868119  2.41868119 ... -0.50610148 -0.50610148
 -0.50610148], shape=(1437,), dtype=float64) tf.Tensor([0.         2.58145359 2.58145359 ... 1.54784909 1.54784909 1.54784909], shape=(1437,), dtype=float64) tf.Tensor(
[ 0.         -2.81943994 -2.81943994 ...  2.92518198  2.92518198
  2.92518198], shape=(1437,), dtype=float64)
tf.Tensor([ 1.5707965  2.4186802  2.41868   ... -0.5060998 -0.5060998 -0.5060998], shape=(1437,), dtype=float32) tf.Tensor([1.5707965 2.58145   2.58145   ... 1.5478499 1.5478499 1.5478499], shape=(1437,), dtype=float32) tf.Tensor([-1.5707965 -2.8194404 -2.8194401 ...  2.9251804  2.9251804  2.92518  ], shape=(1437,), dtype=float32)


In [183]:
th_data =  tf.cast(
    th_data, position_data.dtype, name=None
)
position_ = robot.get_position(pos,th_data)

In [15]:
len(np.where(pos_[:,1,1] <0 )[0])

NameError: name 'pos_' is not defined

In [7]:
robot.th_data = th_data[1:]
robot.position_data = position_data[1:]

In [277]:
#robot = UR3_DNN(500)

#robot.th_data = th_data[1:]
try:
    with tf.device('/device:GPU:1'):
        
        #robot.th_data = robot.th_sample
        #robot.th_data = th_data
        
        robot.th_data = tf.convert_to_tensor(
            robot.th_data, dtype=None, dtype_hint=None, name=None
        )

        robot.batch_size= len(robot.th_data)
        pos = robot.HTrans_tf(robot.th_data)
        robot.batch_size= 500

        position_quat =robot.get_pos_data(pos,robot.th_data)
    
        robot.position_data = quat_to_yzy_euler(position_quat)
except RuntimeError as e:
    pass

(460002, 3)


In [278]:
try:
    with tf.device('/device:GPU:1'):
        robot.th_data, robot.position_data = robot.filter_data(robot.th_data,robot.position_data)

        robot.th_data =  tf.cast(
            robot.th_data, robot.position_data.dtype, name=None
        )

        robot.position_data = robot.position_data.numpy()
        robot.th_data = robot.th_data.numpy()
        
except RuntimeError as e:
    pass

In [279]:
try:
    with tf.device('/device:GPU:1'):        
        #robot.th_data = robot.th_data[np.where((robot.position_data[:,0]>0) & (robot.position_data[:,1]>0) & (robot.position_data[:,2]>0)) ]
        robot.th_data = robot.th_data[np.where((robot.position_data[:,2]>-20)) ]
        #robot.position_data = robot.position_data[np.where((robot.position_data[:,0]>0) & (robot.position_data[:,1]>0) & (robot.position_data[:,2]>0)) ]
        robot.position_data = robot.position_data[np.where((robot.position_data[:,2]>-20)) ]
except RuntimeError as e:
    pass

In [226]:
np.where((robot.position_data[:,0]>0) & (robot.position_data[:,1]>0 & (robot.position_data[:,2]>0)))[0][:10]
np.where((robot.position_data[:,2]>0))[0][:10]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10])

In [227]:
len(robot.th_data)

399130

In [228]:
len(robot.position_data)

399130

In [104]:
n = 101500
print(robot.th_data[n])
po_  =robot.th_data[n:n+2].T

print(robot.HTrans(po_,0))

print(robot.position_data[n])

[-1.9674424  -0.5792581   0.8744719   0.87149054  2.4292653   1.3290259 ]
[[ 0.22806932  0.55874597  0.79736273  8.89558155]
 [ 0.94957887 -0.30860269 -0.05535661 34.27511925]
 [ 0.215138    0.76978394 -0.60095618 14.04519963]
 [ 0.          0.          0.          1.        ]]
[ 8.895579   34.27512    14.0452      2.1986601   1.8845202  -0.05823016
 40.        ]


In [280]:
print(robot.position_data[18000])
print(robot.th_data[18000])

[-15.621099     0.94031334  18.402435     1.8959206    0.8491118
  -1.4765645   40.        ]
[-0.9900476 -0.3901283  2.5497732  1.626616   1.4102603  2.5347497]


In [46]:
print(robot.position_data[18000])
print(robot.th_data[18000])

[20.446766    5.60987    33.832443    2.4804497   0.37342992  1.4900204
 60.        ]
[ 2.7334368   0.007451   -2.290438    0.34758478 -0.38930368  2.0170383 ]


In [32]:
len(robot.position_data)
len(robot.th_data)

181643

In [281]:
robot.position_data[:,3:6] = robot.position_data[:,3:6]*10 


In [17]:
temx = robot.position_data
temy = robot.th_data

In [20]:
robot.position_data = np.vstack([temx,robot.position_data[1:]])
robot.th_data = np.vstack([temy,robot.th_data[1:]])

In [235]:
print(len(robot.position_data))
print(len(robot.th_data))

399130
399130


In [238]:
from sklearn.model_selection import train_test_split

try:
    with tf.device('/device:GPU:1'):

        def data_split(data_y,data):
            x_train, x_test, y_train, y_test = train_test_split(data_y, data, test_size=0.2, shuffle=True)#데이터셋 전처리
            x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test, test_size=0.5, shuffle=True)#데이터셋 전처리
            return x_train,y_train,x_test,y_test,x_valid,y_valid

        def get_weight_set_weight(M,name):
            trained_model = tf.keras.models.load_model(name,compile = False)
            a = trained_model.get_weights()
            M.set_weights(a)
            return M

        def train_model(model,x_train,y_train,x_valid,y_valid,count_,name):

            tf.debugging.set_log_device_placement(True)
            try:
                with tf.device('/device:GPU:1'):
                    # model.compile(optimizer=Adamax(learning_rate=learning_rate), loss='mean_squared_error',metrics = [tf.keras.metrics.Accuracy()])
                    #model.compile(optimizer=Adamax(learning_rate=learning_rate), loss='mse',metrics = [tf.keras.metrics.MeanAbsoluteError()])

                    opt_adamw = tfa.optimizers.AdamW(learning_rate = learning_rate, weight_decay = learning_rate*0.01)
                    model.compile(optimizer=opt_adamw, loss='mse',metrics = [tf.keras.metrics.MeanAbsoluteError()])

                    if count_%10==0:
                        checkpoint = ModelCheckpoint(name,monitor='loss',verbose=0,save_best_only=True,mode='auto')
                    else:
                        checkpoint = ModelCheckpoint(name,monitor='loss',verbose=0,save_best_only=True,mode='auto')
                    #es = EarlyStopping(monitor = 'val_loss',mode='auto',patience = 10)
                    hist = model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_valid,y_valid),callbacks=[checkpoint])
                    model.save('last_'+name)
            except RuntimeError as e:
                pass
            return hist

        def evaluation():
            score=model.evaluate(x_tst,y_tst,verbose=0)
            print('테스트 손실값 ',score[0])
            print('테스트 정확도 ',score[1])
        from numpy.linalg import norm    


        #x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(position_data[1:,:] ,th_data[1:,:])
        
except RuntimeError as e:
    pass


In [239]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.optimizers import Adamax,Adam, Nadam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.models import Model, load_model, save_model
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


In [191]:
try:
    with tf.device('/device:GPU:1'):

        input_size = len(robot.position_data[1])
        output_size = 6
        node_num = 1000
        learning_rate = 0.001
        epochs = 1500
        batch_size= 500
        count = 0

        conf = 1
        import tensorflow_addons as tfa



        for i in range(1):
            #train_x, x_test, train_y, y_test = train_test_split(position_data[1:],th_data[1:], test_size=0.001, shuffle=True)

            ## 데이터 섞으려고 할 경우 아래 코드 

            inputA = Input(shape=input_size, name='Input states')

            A = Dense(node_num,  name='layer1',kernel_initializer = 'he_normal')(inputA)
            A = tfa.activations.mish(A)
            A = tf.keras.layers.BatchNormalization()(A)

            for i in range(2,19):
                A = Dense(node_num,  name='layer{}'.format(i),kernel_initializer = 'he_normal')(A)
                A = tfa.activations.mish(A)
                A = tf.keras.layers.BatchNormalization()(A)

            A = Dense(int(node_num/2),  name='layer{}'.format(i+1),kernel_initializer = 'he_normal')(A)
            A = tfa.activations.mish(A)
            A = tf.keras.layers.BatchNormalization()(A)

            A = Dense(int(node_num/4),  name='layer{}'.format(i+2),kernel_initializer = 'he_normal')(A)
            A = tfa.activations.mish(A)
            A = tf.keras.layers.BatchNormalization()(A)
            outputs = Dense(output_size, activation='linear', name='Outputs')(A)

            model = Model(inputs= inputA , outputs = outputs)
            model.summary()
except RuntimeError as e:
    pass

        

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input states (InputLayer)       [(None, 7)]          0                                            
__________________________________________________________________________________________________
layer1 (Dense)                  (None, 1000)         8000        Input states[0][0]               
__________________________________________________________________________________________________
tf.convert_to_tensor_120 (TFOpL (None, 1000)         0           layer1[0][0]                     
__________________________________________________________________________________________________
tf.math.softplus_120 (TFOpLambd (None, 1000)         0           tf.convert_to_tensor_120[0][0]   
____________________________________________________________________________________________

In [41]:
try:
    with tf.device('/device:GPU:1'):
        train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.01, shuffle=True)
        train_x, x_test, train_y, y_test = train_test_split(train_datax,train_datay, test_size=0.001, shuffle=True) 
        
        x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_x ,train_y)
        def mish(x):
            return x * K.tanh(K.softplus(x))

        input_size = len(position_data[1])
        output_size = 6
        node_num = 1000
        learning_rate = 0.0002
        epochs = 300
        batch_size= 500
        count = 0

        for i in range(10):

            model = get_weight_set_weight(model,'UR3_DNN_quat.h5')

            hist = train_model(model,x_tr ,y_tr,x_vld,y_vld,count,'UR3_DNN_quat.h5')
            train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.001, shuffle=True)
            x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_datax ,train_datay)
except RuntimeError as e:
    pass


Epoch 1/300
56/56 [==============================] - 7s 37ms/step - loss: 74.2477 - mean_absolute_error: 7.4897 - val_loss: 65.2741 - val_mean_absolute_error: 6.9258
Epoch 2/300
56/56 [==============================] - 1s 26ms/step - loss: 65.2539 - mean_absolute_error: 6.9419 - val_loss: 57.1018 - val_mean_absolute_error: 6.3526


KeyboardInterrupt: 

In [121]:
robot.position_data[1000]

array([ 11.408034,   7.842409, -25.733667,   2.023344,  12.569387,
        25.817875,  70.      ], dtype=float32)

In [245]:
np.min(robot.th_data[1000])

0.38173378

## output data  를 rad에서 deg로 변경 후 학습

In [187]:
robot.position_data[:,3:6] *= 10

In [282]:
robot.th_data = np.rad2deg(robot.th_data)

In [283]:
robot.th_data /= 18

In [284]:
for i in range(len(robot.th_data[1])):
    robot.th_data[:,i] = np.where(robot.th_data[:,i]<0, robot.th_data[:,i]+20 , robot.th_data[:,i])


In [285]:
robot.th_data[100]

array([11.00651 ,  1.924887,  9.698446, 10.088684,  7.534876,  6.908946],
      dtype=float32)

In [ ]:
try:
    with tf.device('/device:GPU:1'):
        train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.01, shuffle=True)
        train_x, x_test, train_y, y_test = train_test_split(train_datax,train_datay, test_size=0.001, shuffle=True) 
        
        x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_x ,train_y)
        def mish(x):
            return x * K.tanh(K.softplus(x))

        input_size = len(position_data[1])
        output_size = 6
        node_num = 1000
        learning_rate = 0.00003
        epochs = 300
        batch_size= 500
        count = 0

        for i in range(10):

            model = get_weight_set_weight(model,'UR3_DNN_quat_deg.h5')

            hist = train_model(model,x_tr ,y_tr,x_vld,y_vld,count,'UR3_DNN_quat_deg.h5')
            train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.001, shuffle=True)
            x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_datax ,train_datay)
except RuntimeError as e:
    pass


In [212]:
try:
    with tf.device('/device:GPU:1'):
        train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.01, shuffle=True)
        train_x, x_test, train_y, y_test = train_test_split(train_datax,train_datay, test_size=0.001, shuffle=True) 
        
        x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_x ,train_y)
        def mish(x):
            return x * K.tanh(K.softplus(x))

        input_size = len(robot.position_data[1])
        output_size = 6
        node_num = 1000
        learning_rate = 0.001
        epochs = 400
        batch_size= 500
        count = 0

        for i in range(30):
            
            if i<5:
                learning_rate = 0.00005
            elif i >= 5 and i < 10:
                learning_rate = 0.00003
            elif i>=10 and i< 15:    
                learning_rate = 0.00002
            else:
                learning_rate = 0.00001
                
            model = get_weight_set_weight(model,'UR3_DNN_deg_1000.h5')
            
            print(learning_rate)
            hist = train_model(model,x_tr ,y_tr,x_vld,y_vld,count,'UR3_DNN_deg_1000.h5')
            train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.001, shuffle=True)
            x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_datax ,train_datay)
except RuntimeError as e:
    pass


5e-05
Epoch 1/400
1011/1011 [==============================] - 30s 25ms/step - loss: 2.5420 - mean_absolute_error: 0.7520 - val_loss: 2.5143 - val_mean_absolute_error: 0.6588
Epoch 2/400
1011/1011 [==============================] - 25s 25ms/step - loss: 2.4723 - mean_absolute_error: 0.7517 - val_loss: 2.8215 - val_mean_absolute_error: 0.6651
Epoch 3/400
1011/1011 [==============================] - 25s 25ms/step - loss: 2.4491 - mean_absolute_error: 0.7481 - val_loss: 2.5484 - val_mean_absolute_error: 0.6627
Epoch 4/400
1011/1011 [==============================] - 25s 25ms/step - loss: 2.4198 - mean_absolute_error: 0.7436 - val_loss: 2.7088 - val_mean_absolute_error: 0.6706
Epoch 5/400
1011/1011 [==============================] - 25s 25ms/step - loss: 2.4309 - mean_absolute_error: 0.7453 - val_loss: 3.3790 - val_mean_absolute_error: 0.6746
Epoch 6/400
1011/1011 [==============================] - 25s 25ms/step - loss: 2.4342 - mean_absolute_error: 0.7480 - val_loss: 2.9055 - val_mean_abs

1011/1011 [==============================] - 25s 25ms/step - loss: 1.9993 - mean_absolute_error: 0.6627 - val_loss: 9.3239 - val_mean_absolute_error: 1.0757
Epoch 50/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9903 - mean_absolute_error: 0.6608 - val_loss: 8.9079 - val_mean_absolute_error: 1.0808
Epoch 51/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9895 - mean_absolute_error: 0.6599 - val_loss: 8.4439 - val_mean_absolute_error: 1.0652
Epoch 52/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9762 - mean_absolute_error: 0.6566 - val_loss: 8.7325 - val_mean_absolute_error: 1.0722
Epoch 53/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9965 - mean_absolute_error: 0.6612 - val_loss: 8.6974 - val_mean_absolute_error: 1.0738
Epoch 54/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9894 - mean_absolute_error: 0.6596 - val_loss: 9.7331 - val_mean_absolute_error: 

1011/1011 [==============================] - 25s 25ms/step - loss: 1.9676 - mean_absolute_error: 0.6524 - val_loss: 11.6706 - val_mean_absolute_error: 1.0910
Epoch 98/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9602 - mean_absolute_error: 0.6498 - val_loss: 13.4680 - val_mean_absolute_error: 1.0909
Epoch 99/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9700 - mean_absolute_error: 0.6527 - val_loss: 19.4138 - val_mean_absolute_error: 1.1073
Epoch 100/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9621 - mean_absolute_error: 0.6471 - val_loss: 16.8504 - val_mean_absolute_error: 1.1187
Epoch 101/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9644 - mean_absolute_error: 0.6512 - val_loss: 15.6226 - val_mean_absolute_error: 1.1021
Epoch 102/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9762 - mean_absolute_error: 0.6518 - val_loss: 14.3644 - val_mean_absolut

Epoch 145/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9609 - mean_absolute_error: 0.6489 - val_loss: 8.2508 - val_mean_absolute_error: 1.0928
Epoch 146/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9556 - mean_absolute_error: 0.6467 - val_loss: 9.0498 - val_mean_absolute_error: 1.1056
Epoch 147/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9603 - mean_absolute_error: 0.6464 - val_loss: 50.5839 - val_mean_absolute_error: 1.1148
Epoch 148/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9501 - mean_absolute_error: 0.6439 - val_loss: 8.2917 - val_mean_absolute_error: 1.1018
Epoch 149/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9506 - mean_absolute_error: 0.6449 - val_loss: 8.1173 - val_mean_absolute_error: 1.0921
Epoch 150/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9506 - mean_absolute_error: 0.6428 - val_loss: 8.0332 - val_m

Epoch 193/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9578 - mean_absolute_error: 0.6458 - val_loss: 8.1812 - val_mean_absolute_error: 1.0979
Epoch 194/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9654 - mean_absolute_error: 0.6456 - val_loss: 7.8657 - val_mean_absolute_error: 1.1169
Epoch 195/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9622 - mean_absolute_error: 0.6486 - val_loss: 8.1063 - val_mean_absolute_error: 1.1041
Epoch 196/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9504 - mean_absolute_error: 0.6467 - val_loss: 8.5022 - val_mean_absolute_error: 1.1121
Epoch 197/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9762 - mean_absolute_error: 0.6462 - val_loss: 8.4113 - val_mean_absolute_error: 1.1210
Epoch 198/400
1011/1011 [==============================] - 25s 25ms/step - loss: 1.9384 - mean_absolute_error: 0.6394 - val_loss: 7.8372 - val_me

...

Epoch 177/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9565 - mean_absolute_error: 0.6385 - val_loss: 7.8008 - val_mean_absolute_error: 1.0685
Epoch 178/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9696 - mean_absolute_error: 0.6389 - val_loss: 7.7436 - val_mean_absolute_error: 1.0628
Epoch 179/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9597 - mean_absolute_error: 0.6401 - val_loss: 7.9253 - val_mean_absolute_error: 1.0742
Epoch 180/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9438 - mean_absolute_error: 0.6343 - val_loss: 8.0608 - val_mean_absolute_error: 1.0716
Epoch 181/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9630 - mean_absolute_error: 0.6396 - val_loss: 7.7958 - val_mean_absolute_error: 1.0737
Epoch 182/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9556 - mean_absolute_error: 0.6356 - val_loss: 8.0747 - val_me

Epoch 225/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9648 - mean_absolute_error: 0.6377 - val_loss: 7.9151 - val_mean_absolute_error: 1.0870
Epoch 226/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9454 - mean_absolute_error: 0.6336 - val_loss: 7.9686 - val_mean_absolute_error: 1.0797
Epoch 227/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9509 - mean_absolute_error: 0.6350 - val_loss: 7.9105 - val_mean_absolute_error: 1.0852
Epoch 228/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9460 - mean_absolute_error: 0.6340 - val_loss: 7.8063 - val_mean_absolute_error: 1.0750
Epoch 229/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9686 - mean_absolute_error: 0.6390 - val_loss: 7.8590 - val_mean_absolute_error: 1.0779
Epoch 230/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9460 - mean_absolute_error: 0.6349 - val_loss: 7.8441 - val_me

Epoch 273/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9489 - mean_absolute_error: 0.6327 - val_loss: 7.8640 - val_mean_absolute_error: 1.0815
Epoch 274/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9495 - mean_absolute_error: 0.6341 - val_loss: 7.8950 - val_mean_absolute_error: 1.0830
Epoch 275/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9474 - mean_absolute_error: 0.6323 - val_loss: 7.8744 - val_mean_absolute_error: 1.0815
Epoch 276/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9540 - mean_absolute_error: 0.6319 - val_loss: 7.8845 - val_mean_absolute_error: 1.0793
Epoch 277/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9448 - mean_absolute_error: 0.6306 - val_loss: 7.8424 - val_mean_absolute_error: 1.0759
Epoch 278/400
1021/1021 [==============================] - 25s 25ms/step - loss: 1.9420 - mean_absolute_error: 0.6293 - val_loss: 7.8598 - val_me

Epoch 321/400
 717/1021 [====================>.........] - ETA: 7s - loss: 1.9542 - mean_absolute_error: 0.6344

KeyboardInterrupt: 

In [249]:
try:
    with tf.device('/device:GPU:1'):
        train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.01, shuffle=True)
        train_x, x_test, train_y, y_test = train_test_split(train_datax,train_datay, test_size=0.001, shuffle=True) 
        
        x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_x ,train_y)
        def mish(x):
            return x * K.tanh(K.softplus(x))

        input_size = len(robot.position_data[1])
        output_size = 6
        node_num = 1000
        learning_rate = 0.001
        epochs = 400
        batch_size= 500
        count = 0
    
        #model = get_weight_set_weight(model,'UR3_DNN_deg_1000.h5')
        
        for i in range(10):
            
            if i<5:
                learning_rate = 0.00001
            elif i >= 5 and i < 10:
                learning_rate = 0.000005
            elif i>=10 and i< 15:    
                learning_rate = 0.000005
            else:
                learning_rate = 0.000002
                
            
            
            print(learning_rate)
            hist = train_model(model,x_tr[:int(len(x_tr)/2)] ,y_tr[:int(len(x_tr)/2)],x_vld[:int(len(x_vld)/2)],y_vld[:int(len(x_vld)/2)],count,'UR3_DNN_deg_1000_2.h5')
            train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.001, shuffle=True)
            x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_datax ,train_datay)
except RuntimeError as e:
    pass


1e-05
Epoch 1/400
316/316 [==============================] - 13s 27ms/step - loss: 3.1683 - mean_absolute_error: 0.7624 - val_loss: 2.7126 - val_mean_absolute_error: 0.6820
Epoch 2/400
316/316 [==============================] - 8s 25ms/step - loss: 2.5260 - mean_absolute_error: 0.7425 - val_loss: 2.7262 - val_mean_absolute_error: 0.6730
Epoch 3/400
316/316 [==============================] - 8s 25ms/step - loss: 2.4577 - mean_absolute_error: 0.7384 - val_loss: 2.8397 - val_mean_absolute_error: 0.6813
Epoch 4/400
316/316 [==============================] - 8s 25ms/step - loss: 2.3443 - mean_absolute_error: 0.7234 - val_loss: 3.0385 - val_mean_absolute_error: 0.6938
Epoch 5/400
316/316 [==============================] - 8s 25ms/step - loss: 2.1733 - mean_absolute_error: 0.6866 - val_loss: 3.3022 - val_mean_absolute_error: 0.7141
Epoch 6/400
316/316 [==============================] - 8s 25ms/step - loss: 2.0162 - mean_absolute_error: 0.6616 - val_loss: 3.6652 - val_mean_absolute_error: 0.73

316/316 [==============================] - 8s 25ms/step - loss: 1.1013 - mean_absolute_error: 0.4560 - val_loss: 5.7249 - val_mean_absolute_error: 0.8745
Epoch 51/400
316/316 [==============================] - 8s 25ms/step - loss: 1.1059 - mean_absolute_error: 0.4629 - val_loss: 5.7208 - val_mean_absolute_error: 0.8735
Epoch 52/400
316/316 [==============================] - 8s 25ms/step - loss: 1.1137 - mean_absolute_error: 0.4606 - val_loss: 5.7382 - val_mean_absolute_error: 0.8777
Epoch 53/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0880 - mean_absolute_error: 0.4548 - val_loss: 5.7364 - val_mean_absolute_error: 0.8831
Epoch 54/400
316/316 [==============================] - 8s 25ms/step - loss: 1.1076 - mean_absolute_error: 0.4594 - val_loss: 5.7486 - val_mean_absolute_error: 0.8796
Epoch 55/400
316/316 [==============================] - 8s 25ms/step - loss: 1.1076 - mean_absolute_error: 0.4636 - val_loss: 5.7090 - val_mean_absolute_error: 0.8754
Epoch 56/40

316/316 [==============================] - 8s 25ms/step - loss: 1.0729 - mean_absolute_error: 0.4479 - val_loss: 5.7819 - val_mean_absolute_error: 0.8796
Epoch 100/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0661 - mean_absolute_error: 0.4470 - val_loss: 5.7645 - val_mean_absolute_error: 0.8820
Epoch 101/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0692 - mean_absolute_error: 0.4454 - val_loss: 5.7911 - val_mean_absolute_error: 0.8820
Epoch 102/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0609 - mean_absolute_error: 0.4416 - val_loss: 5.7918 - val_mean_absolute_error: 0.8817
Epoch 103/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0674 - mean_absolute_error: 0.4500 - val_loss: 5.7821 - val_mean_absolute_error: 0.8854
Epoch 104/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0676 - mean_absolute_error: 0.4499 - val_loss: 5.7622 - val_mean_absolute_error: 0.8804
Epoch 

Epoch 148/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0511 - mean_absolute_error: 0.4393 - val_loss: 5.7799 - val_mean_absolute_error: 0.8870
Epoch 149/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0529 - mean_absolute_error: 0.4394 - val_loss: 5.7910 - val_mean_absolute_error: 0.8875
Epoch 150/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0553 - mean_absolute_error: 0.4437 - val_loss: 5.8100 - val_mean_absolute_error: 0.8860
Epoch 151/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0352 - mean_absolute_error: 0.4393 - val_loss: 5.7881 - val_mean_absolute_error: 0.8803
Epoch 152/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0404 - mean_absolute_error: 0.4418 - val_loss: 5.8130 - val_mean_absolute_error: 0.8856
Epoch 153/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0394 - mean_absolute_error: 0.4364 - val_loss: 5.7785 - val_mean_absolute_error:

Epoch 197/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0308 - mean_absolute_error: 0.4304 - val_loss: 5.8053 - val_mean_absolute_error: 0.8822
Epoch 198/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0553 - mean_absolute_error: 0.4384 - val_loss: 5.8179 - val_mean_absolute_error: 0.8838
Epoch 199/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0449 - mean_absolute_error: 0.4370 - val_loss: 5.7944 - val_mean_absolute_error: 0.8835
Epoch 200/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0344 - mean_absolute_error: 0.4349 - val_loss: 5.8195 - val_mean_absolute_error: 0.8941
Epoch 201/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0520 - mean_absolute_error: 0.4401 - val_loss: 5.8082 - val_mean_absolute_error: 0.8837
Epoch 202/400
316/316 [==============================] - 8s 25ms/step - loss: 1.0223 - mean_absolute_error: 0.4282 - val_loss: 5.7882 - val_mean_absolute_error:

...

Epoch 174/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0873 - mean_absolute_error: 0.4500 - val_loss: 5.6141 - val_mean_absolute_error: 0.8524
Epoch 175/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0878 - mean_absolute_error: 0.4527 - val_loss: 5.6262 - val_mean_absolute_error: 0.8584
Epoch 176/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0752 - mean_absolute_error: 0.4485 - val_loss: 5.6222 - val_mean_absolute_error: 0.8580
Epoch 177/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0979 - mean_absolute_error: 0.4522 - val_loss: 5.6019 - val_mean_absolute_error: 0.8536
Epoch 178/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0729 - mean_absolute_error: 0.4403 - val_loss: 5.6510 - val_mean_absolute_error: 0.8567
Epoch 179/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0863 - mean_absolute_error: 0.4506 - val_loss: 5.6316 - val_mean_absolute_error:

Epoch 223/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0725 - mean_absolute_error: 0.4457 - val_loss: 5.6191 - val_mean_absolute_error: 0.8542
Epoch 224/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0481 - mean_absolute_error: 0.4369 - val_loss: 5.6016 - val_mean_absolute_error: 0.8567
Epoch 225/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0728 - mean_absolute_error: 0.4461 - val_loss: 5.6205 - val_mean_absolute_error: 0.8531
Epoch 226/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0722 - mean_absolute_error: 0.4430 - val_loss: 5.6230 - val_mean_absolute_error: 0.8555
Epoch 227/400
319/319 [==============================] - 8s 24ms/step - loss: 1.0630 - mean_absolute_error: 0.4423 - val_loss: 5.6213 - val_mean_absolute_error: 0.8562
Epoch 228/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0662 - mean_absolute_error: 0.4398 - val_loss: 5.6151 - val_mean_absolute_error:

Epoch 272/400
319/319 [==============================] - 8s 24ms/step - loss: 1.0575 - mean_absolute_error: 0.4398 - val_loss: 5.6233 - val_mean_absolute_error: 0.8547
Epoch 273/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0696 - mean_absolute_error: 0.4431 - val_loss: 5.6387 - val_mean_absolute_error: 0.8531
Epoch 274/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0675 - mean_absolute_error: 0.4437 - val_loss: 5.6242 - val_mean_absolute_error: 0.8571
Epoch 275/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0700 - mean_absolute_error: 0.4417 - val_loss: 5.6364 - val_mean_absolute_error: 0.8550
Epoch 276/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0603 - mean_absolute_error: 0.4410 - val_loss: 5.6354 - val_mean_absolute_error: 0.8556
Epoch 277/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0775 - mean_absolute_error: 0.4484 - val_loss: 5.6304 - val_mean_absolute_error:

Epoch 321/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0728 - mean_absolute_error: 0.4496 - val_loss: 5.6453 - val_mean_absolute_error: 0.8560
Epoch 322/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0678 - mean_absolute_error: 0.4439 - val_loss: 5.6384 - val_mean_absolute_error: 0.8551
Epoch 323/400
319/319 [==============================] - 8s 24ms/step - loss: 1.0615 - mean_absolute_error: 0.4417 - val_loss: 5.6274 - val_mean_absolute_error: 0.8573
Epoch 324/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0475 - mean_absolute_error: 0.4345 - val_loss: 5.6472 - val_mean_absolute_error: 0.8574
Epoch 325/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0676 - mean_absolute_error: 0.4440 - val_loss: 5.6251 - val_mean_absolute_error: 0.8534
Epoch 326/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0596 - mean_absolute_error: 0.4401 - val_loss: 5.6035 - val_mean_absolute_error:

Epoch 370/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0605 - mean_absolute_error: 0.4390 - val_loss: 5.6385 - val_mean_absolute_error: 0.8541
Epoch 371/400
319/319 [==============================] - 8s 24ms/step - loss: 1.0551 - mean_absolute_error: 0.4403 - val_loss: 5.6135 - val_mean_absolute_error: 0.8525
Epoch 372/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0492 - mean_absolute_error: 0.4370 - val_loss: 5.6376 - val_mean_absolute_error: 0.8553
Epoch 373/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0590 - mean_absolute_error: 0.4362 - val_loss: 5.6288 - val_mean_absolute_error: 0.8574
Epoch 374/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0829 - mean_absolute_error: 0.4499 - val_loss: 5.6524 - val_mean_absolute_error: 0.8596
Epoch 375/400
319/319 [==============================] - 8s 25ms/step - loss: 1.0463 - mean_absolute_error: 0.4430 - val_loss: 5.6325 - val_mean_absolute_error:

In [292]:
try:
    with tf.device('/device:GPU:1'):
        train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.01, shuffle=True)
        train_x, x_test, train_y, y_test = train_test_split(train_datax,train_datay, test_size=0.001, shuffle=True) 
        
        x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_x ,train_y)
        def mish(x):
            return x * K.tanh(K.softplus(x))

        input_size = len(robot.position_data[1])
        output_size = 6
        node_num = 1000
        learning_rate = 0.001
        epochs = 200
        batch_size= 1000
        count = 0
    
        model = get_weight_set_weight(model,'UR3_DNN_deg_1000_3.h5')
        
        for i in range(20):
            
            if i<5:
                learning_rate = 0.00005
            elif i >= 5 and i < 10:
                learning_rate = 0.00003
            elif i>=10 and i< 15:    
                learning_rate = 0.000005
            else:
                learning_rate = 0.000002
                
            
            
            print(learning_rate)
            hist = train_model(model,x_tr[:int(len(x_tr)/3)] ,y_tr[:int(len(x_tr)/3)],x_vld[:int(len(x_vld)/2)],y_vld[:int(len(x_vld)/2)],count,'UR3_DNN_deg_1000_3.h5')
            train_datax, x_test, train_datay, y_test = train_test_split(robot.position_data[1:],robot.th_data[1:], test_size=0.001, shuffle=True)
            x_tr,y_tr,x_tst,y_tst,x_vld,y_vld = data_split(train_datax ,train_datay)
except RuntimeError as e:
    pass


5e-05
Epoch 1/200
114/114 [==============================] - 9s 38ms/step - loss: 3.5108 - mean_absolute_error: 0.7795 - val_loss: 3.4237 - val_mean_absolute_error: 0.8083
Epoch 2/200
114/114 [==============================] - 4s 34ms/step - loss: 2.6560 - mean_absolute_error: 0.7499 - val_loss: 3.3962 - val_mean_absolute_error: 0.7638
Epoch 3/200
114/114 [==============================] - 4s 34ms/step - loss: 2.3728 - mean_absolute_error: 0.7067 - val_loss: 3.6815 - val_mean_absolute_error: 0.7755
Epoch 4/200
114/114 [==============================] - 4s 34ms/step - loss: 2.0516 - mean_absolute_error: 0.6518 - val_loss: 4.0439 - val_mean_absolute_error: 0.8052
Epoch 5/200
114/114 [==============================] - 4s 34ms/step - loss: 1.7613 - mean_absolute_error: 0.5954 - val_loss: 4.2583 - val_mean_absolute_error: 0.8300
Epoch 6/200
114/114 [==============================] - 4s 34ms/step - loss: 1.5320 - mean_absolute_error: 0.5458 - val_loss: 4.6043 - val_mean_absolute_error: 0.831

114/114 [==============================] - 4s 35ms/step - loss: 0.7032 - mean_absolute_error: 0.3453 - val_loss: 5.5110 - val_mean_absolute_error: 0.8680
Epoch 51/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7100 - mean_absolute_error: 0.3404 - val_loss: 5.5043 - val_mean_absolute_error: 0.8616
Epoch 52/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7168 - mean_absolute_error: 0.3405 - val_loss: 5.4389 - val_mean_absolute_error: 0.8742
Epoch 53/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7251 - mean_absolute_error: 0.3499 - val_loss: 5.6528 - val_mean_absolute_error: 0.8705
Epoch 54/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7194 - mean_absolute_error: 0.3435 - val_loss: 5.4221 - val_mean_absolute_error: 0.8709
Epoch 55/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7058 - mean_absolute_error: 0.3375 - val_loss: 5.4772 - val_mean_absolute_error: 0.8729
Epoch 56/20

114/114 [==============================] - 4s 34ms/step - loss: 0.6790 - mean_absolute_error: 0.3288 - val_loss: 5.8499 - val_mean_absolute_error: 0.8783
Epoch 100/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6851 - mean_absolute_error: 0.3315 - val_loss: 6.0625 - val_mean_absolute_error: 0.8775
Epoch 101/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6947 - mean_absolute_error: 0.3335 - val_loss: 5.8153 - val_mean_absolute_error: 0.8786
Epoch 102/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6872 - mean_absolute_error: 0.3271 - val_loss: 5.5567 - val_mean_absolute_error: 0.8779
Epoch 103/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7021 - mean_absolute_error: 0.3331 - val_loss: 5.6917 - val_mean_absolute_error: 0.8710
Epoch 104/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6843 - mean_absolute_error: 0.3252 - val_loss: 5.5687 - val_mean_absolute_error: 0.8684
Epoch 

Epoch 148/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6757 - mean_absolute_error: 0.3122 - val_loss: 5.2421 - val_mean_absolute_error: 0.8748
Epoch 149/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6829 - mean_absolute_error: 0.3155 - val_loss: 5.1828 - val_mean_absolute_error: 0.8831
Epoch 150/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6746 - mean_absolute_error: 0.3214 - val_loss: 5.1804 - val_mean_absolute_error: 0.8655
Epoch 151/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6586 - mean_absolute_error: 0.3107 - val_loss: 6.1225 - val_mean_absolute_error: 0.8692
Epoch 152/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6698 - mean_absolute_error: 0.3166 - val_loss: 5.6350 - val_mean_absolute_error: 0.8659
Epoch 153/200
114/114 [==============================] - 4s 34ms/step - loss: 0.6639 - mean_absolute_error: 0.3092 - val_loss: 5.3982 - val_mean_absolute_error:

Epoch 197/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7112 - mean_absolute_error: 0.3399 - val_loss: 5.2981 - val_mean_absolute_error: 0.9080
Epoch 198/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7001 - mean_absolute_error: 0.3295 - val_loss: 5.3180 - val_mean_absolute_error: 0.8941
Epoch 199/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7144 - mean_absolute_error: 0.3387 - val_loss: 5.3245 - val_mean_absolute_error: 0.9024
Epoch 200/200
114/114 [==============================] - 4s 34ms/step - loss: 0.7001 - mean_absolute_error: 0.3272 - val_loss: 5.3468 - val_mean_absolute_error: 0.9000
5e-05
Epoch 1/200
115/115 [==============================] - 9s 40ms/step - loss: 3.5528 - mean_absolute_error: 0.7918 - val_loss: 3.1700 - val_mean_absolute_error: 0.8133
Epoch 2/200
115/115 [==============================] - 4s 34ms/step - loss: 2.6905 - mean_absolute_error: 0.7612 - val_loss: 3.1036 - val_mean_absolute_erro

...

Epoch 174/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9729 - mean_absolute_error: 0.4389 - val_loss: 4.7736 - val_mean_absolute_error: 0.8007
Epoch 175/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9852 - mean_absolute_error: 0.4458 - val_loss: 4.7852 - val_mean_absolute_error: 0.8011
Epoch 176/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9940 - mean_absolute_error: 0.4420 - val_loss: 4.7756 - val_mean_absolute_error: 0.8006
Epoch 177/200
115/115 [==============================] - 4s 34ms/step - loss: 1.0047 - mean_absolute_error: 0.4484 - val_loss: 4.7719 - val_mean_absolute_error: 0.8002
Epoch 178/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9820 - mean_absolute_error: 0.4397 - val_loss: 4.7748 - val_mean_absolute_error: 0.8005
Epoch 179/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9944 - mean_absolute_error: 0.4425 - val_loss: 4.7870 - val_mean_absolute_error:

Epoch 23/200
115/115 [==============================] - 4s 34ms/step - loss: 1.9930 - mean_absolute_error: 0.6448 - val_loss: 3.1920 - val_mean_absolute_error: 0.7034
Epoch 24/200
115/115 [==============================] - 4s 34ms/step - loss: 1.9177 - mean_absolute_error: 0.6284 - val_loss: 3.2328 - val_mean_absolute_error: 0.7052
Epoch 25/200
115/115 [==============================] - 4s 34ms/step - loss: 1.9419 - mean_absolute_error: 0.6315 - val_loss: 3.2604 - val_mean_absolute_error: 0.7070
Epoch 26/200
115/115 [==============================] - 4s 34ms/step - loss: 1.9138 - mean_absolute_error: 0.6292 - val_loss: 3.2934 - val_mean_absolute_error: 0.7085
Epoch 27/200
115/115 [==============================] - 4s 34ms/step - loss: 1.8951 - mean_absolute_error: 0.6294 - val_loss: 3.3255 - val_mean_absolute_error: 0.7097
Epoch 28/200
115/115 [==============================] - 4s 34ms/step - loss: 1.8502 - mean_absolute_error: 0.6194 - val_loss: 3.3573 - val_mean_absolute_error: 0.711

115/115 [==============================] - 4s 34ms/step - loss: 1.2449 - mean_absolute_error: 0.5052 - val_loss: 4.1526 - val_mean_absolute_error: 0.7551
Epoch 73/200
115/115 [==============================] - 4s 34ms/step - loss: 1.2483 - mean_absolute_error: 0.4990 - val_loss: 4.1574 - val_mean_absolute_error: 0.7552
Epoch 74/200
115/115 [==============================] - 4s 34ms/step - loss: 1.2469 - mean_absolute_error: 0.5022 - val_loss: 4.1738 - val_mean_absolute_error: 0.7563
Epoch 75/200
115/115 [==============================] - 4s 34ms/step - loss: 1.2369 - mean_absolute_error: 0.5061 - val_loss: 4.1810 - val_mean_absolute_error: 0.7571
Epoch 76/200
115/115 [==============================] - 4s 34ms/step - loss: 1.2288 - mean_absolute_error: 0.5036 - val_loss: 4.1881 - val_mean_absolute_error: 0.7573
Epoch 77/200
115/115 [==============================] - 4s 34ms/step - loss: 1.2148 - mean_absolute_error: 0.5000 - val_loss: 4.1877 - val_mean_absolute_error: 0.7573
Epoch 78/20

115/115 [==============================] - 4s 34ms/step - loss: 1.0842 - mean_absolute_error: 0.4662 - val_loss: 4.4398 - val_mean_absolute_error: 0.7712
Epoch 122/200
115/115 [==============================] - 4s 34ms/step - loss: 1.0733 - mean_absolute_error: 0.4734 - val_loss: 4.4462 - val_mean_absolute_error: 0.7719
Epoch 123/200
115/115 [==============================] - 4s 34ms/step - loss: 1.0563 - mean_absolute_error: 0.4618 - val_loss: 4.4540 - val_mean_absolute_error: 0.7723
Epoch 124/200
115/115 [==============================] - 4s 34ms/step - loss: 1.0677 - mean_absolute_error: 0.4627 - val_loss: 4.4521 - val_mean_absolute_error: 0.7723
Epoch 125/200
115/115 [==============================] - 4s 34ms/step - loss: 1.0722 - mean_absolute_error: 0.4599 - val_loss: 4.4504 - val_mean_absolute_error: 0.7720
Epoch 126/200
115/115 [==============================] - 4s 35ms/step - loss: 1.0751 - mean_absolute_error: 0.4612 - val_loss: 4.4479 - val_mean_absolute_error: 0.7723
Epoch 

Epoch 170/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9920 - mean_absolute_error: 0.4453 - val_loss: 4.5491 - val_mean_absolute_error: 0.7769
Epoch 171/200
115/115 [==============================] - 4s 34ms/step - loss: 1.0141 - mean_absolute_error: 0.4474 - val_loss: 4.5537 - val_mean_absolute_error: 0.7769
Epoch 172/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9848 - mean_absolute_error: 0.4391 - val_loss: 4.5409 - val_mean_absolute_error: 0.7764
Epoch 173/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9946 - mean_absolute_error: 0.4480 - val_loss: 4.5580 - val_mean_absolute_error: 0.7777
Epoch 174/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9775 - mean_absolute_error: 0.4457 - val_loss: 4.5582 - val_mean_absolute_error: 0.7772
Epoch 175/200
115/115 [==============================] - 4s 34ms/step - loss: 0.9606 - mean_absolute_error: 0.4366 - val_loss: 4.5532 - val_mean_absolute_error:

In [266]:
last_position_data = robot.position_data
last_th_data = robot.th_data

In [146]:
np.where(robot.position_data[:,2]<0)

(array([    7,     9,    10, ..., 21965, 21969, 21970]),)

In [288]:
print(len(last_position_data))
len(robot.position_data)

399130


430667

## UR3_DNN_quat.h5 
 input : x,y,z,q_x,q_y,q_z,q_w,conf 
 
 output : th1~th6 [rad]
 
## UR3_DNN_quat_deg.h5
 input : x,y,z,q_x,q_y,q_z,q_w,conf
 
 output : th1~th6 [deg / 10]
 
 
 quat * 10
 
  hidden node = 1000
  
  loss = 0.0065
 ## UR3_DNN_deg.h5

 input : x,y,z,a,b,g,conf  a,b,g * 10 
 
 output : th1~th6 [deg / 18/2]   범위는  0 - 10

 hidden node = 750
 
 loss = 0.5
 
 ## UR3_DNN_deg_1000.h5
 
 input : x,y,z,a,b,g,conf  a,b,g * 10  
  output : th1~th6 [deg / 18/2]   범위는  0 - 20

 hidden node = 1000
 loss = 1.9

## UR3_DNN_deg_1000_2.h5
 
 UR3_DNN_deg_1000.h5 에서 dataset만 바꿔서 transfer learning 시킨거 
 
 input : x,y,z,a,b,g,conf  a,b,g * 10  
  output : th1~th6 [deg / 18/2]   범위는  0 - 20

 hidden node = 1000
 
 transfer learning 시킬 때, data set을 split해서 overfitting 시킨 후 다른 데이터셋이랑 섞어서 재 학습시키니 loss가 절반으로 줄어듬.
 
 


In [32]:
x_tst = robot.position_data
y_tst = robot.th_data

In [257]:
def get_4(pos):
    xyz = pos[:3]
    quat = pos[3:7]
    mat_ = from_quaternion(quat)
    
    position = np.eye(4)
    position[:3,:3]=mat_
    position[:3,3]=xyz
    return position

def prediction_deg(n,conf_=0):
    x_ = copy.deepcopy(x_tst[n])

    if conf_<=0 or conf_>8:
        print("데이터 그대로")
    else:
        x_[-1] = conf_*10
        print("conf 변경")

    x_ = np.expand_dims(x_,axis=0)

    pred = trained_model.predict(x_)
    x_[0][3:7] /= 10
    
    #print(x_)

    print('input position : ',x_)
    print(conf_)
    
    y_ = copy.deepcopy(y_tst[n])
    y_ *= 10
    print('answer theta : ',y_)

    
    pred *= 10
    pred_ = np.deg2rad(pred)
    pred_= np.array([pred_]).T

    end_point = HTrans(pred_,0)

    x1 = end_point[0,3]
    y1 = end_point[1,3]
    z1 = end_point[2,3]
    a = (atan2(end_point[2,1],-end_point[0,1]))
    b = (atan2(sqrt(1-end_point[1,1]**2),end_point[1,1] ))
    g = (atan2(end_point[1,2],end_point[1,0]))
    print('pred theta : ',pred)
    print()


    in_pos = get_4(x_[0])
    print(in_pos,"input_pos ")
    print()
    print(end_point,"reconstruct_pos")


    
    error = sqrt(np.sum(np.square(in_pos[:3,3] - end_point[:3,3].T)))

    #print( 'similarity : ',cos_sim(in_pos[0],recon_pos[0])) # 코사인 유사도 평가
    print('RMSE :',RMSE(np.asarray(in_pos),np.asarray(end_point))) # 평균 제곱근 오차
    print("위치 차이 : ",error)
    

    
def prediction_deg_2(model,n,conf_=0):
    x_ = copy.deepcopy(x_tst[n])

    if conf_<=0 or conf_>8:
        print("데이터 그대로")
    else:
        x_[-1] = conf_*10
        print("conf 변경")

    x_ = np.expand_dims(x_,axis=0)

    pred = model.predict(x_)
    x_[0][3:7] /= 10
    #x_[0][3:7] = np.deg2rad(x_[0][3:7])
    
    #print(x_)

    print('input position : ',x_)
    print(conf_)
    
    y_ = copy.deepcopy(y_tst[n])

    y_ *= 18
    y_ = np.deg2rad(y_)
    print('answer theta : ',y_)


    
    pred_ = pred *18
    pred_ = np.deg2rad(pred_)
    pred_= np.array([pred_]).T

    end_point = HTrans(pred_,0)

    x1 = end_point[0,3]
    y1 = end_point[1,3]
    z1 = end_point[2,3]
    a = (atan2(end_point[2,1],-end_point[0,1]))
    b = (atan2(sqrt(1-end_point[1,1]**2),end_point[1,1] ))
    g = (atan2(end_point[1,2],end_point[1,0]))
    print('pred theta : ',pred_.T)
    print()


    in_pos = robot.get_pos(x_[0])
    print(in_pos,"input_pos ")
    print()
    print(end_point,"reconstruct_pos")


    
    error = sqrt(np.sum(np.square(in_pos[:3,3] - end_point[:3,3])))

    #print( 'similarity : ',cos_sim(in_pos[0],recon_pos[0])) # 코사인 유사도 평가
    print('RMSE :',RMSE(np.asarray(in_pos),np.asarray(end_point))) # 평균 제곱근 오차
    print("위치 차이 : ",error)
    


In [261]:
'''
5월 9일
'''
import copy

try:
    with tf.device('/device:GPU:1'):
        trained_model = tf.keras.models.load_model('UR3_DNN_deg_1000.h5',compile = False)
        trained_model2 = tf.keras.models.load_model('UR3_DNN_deg_1000_2.h5',compile = False)
        
        n=152
        
        
        prediction_deg_2(trained_model,n)
        prediction_deg_2(trained_model2,n)
        
except RuntimeError as e:
    pass


데이터 그대로
input position :  [[33.56936     7.63691     9.16754     0.817551    0.08050996  2.441629
   5.        ]]
0
answer theta :  [0.52988094 3.5538366  5.4702463  1.8947198  4.607476   5.179696  ]
pred theta :  [[[1.7596467 2.182026  4.873071  3.9595432 4.670437  3.2744908]]]

[[-0.99139992 -0.05501021 -0.11874374 33.56935883]
 [-0.06151287  0.99676082  0.05180766  7.63690996]
 [ 0.11550915  0.05866638 -0.99157243  9.1675396 ]
 [ 0.          0.          0.          1.        ]] input_pos 

[[ 9.97425188e-01  5.60235437e-02 -4.47700549e-02  1.25143468e+01]
 [ 5.65613301e-02 -9.98340320e-01  1.08360994e-02 -7.45932608e+00]
 [-4.40886742e-02 -1.33404523e-02 -9.98938547e-01 -2.79818973e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] reconstruct_pos
RMSE : 11.344851597947486
위치 차이 :  45.29117506871512
데이터 그대로
input position :  [[33.56936     7.63691     9.16754     0.817551    0.08050996  2.441629
   5.        ]]
0
answer theta :  [0.52988094 3.5538366  5.4702463

In [211]:
'''
conf_1
'''
import copy

try:
    with tf.device('/device:GPU:1'):
        trained_model = tf.keras.models.load_model('UR3_DNN_deg_1000.h5',compile = False)
        n=50
        prediction_deg_2(n)
        
except RuntimeError as e:
    pass


데이터 그대로
input position :  [[ 6.9557705 25.233051  40.83547   -1.4068401  1.8632    -2.06232
   2.       ]]
0
answer theta :  [4.2698164  0.20230964 4.6888986  4.8417244  2.5318568  6.2521205 ]
pred theta :  [[[4.244012   0.21025224 4.642731   4.8790016  2.5312598  6.237818  ]]]

[[-0.84758559 -0.15629459  0.50711998  6.95577002]
 [-0.4519364  -0.28825465 -0.84419355 25.23304939]
 [ 0.27812258 -0.94471226  0.17368528 40.83546829]
 [ 0.          0.          0.          1.        ]] input_pos 

[[-0.85694918 -0.17555901  0.48457934  7.21495851]
 [-0.42554389 -0.28942713 -0.85740558 24.28140515]
 [ 0.29077568 -0.94096279  0.17331628 40.77431482]
 [ 0.          0.          0.          1.        ]] reconstruct_pos
RMSE : 0.2473053352371111
위치 차이 :  58.44147812207758


In [19]:
'''
conf_1
'''
import copy

try:
    with tf.device('/device:GPU:1'):
        trained_model = tf.keras.models.load_model('UR3_DNN_quat_deg.h5',compile = False)
        n=1155
        prediction_deg(n)
        
except RuntimeError as e:
    pass


데이터 그대로
input position :  [[ 3.0885799e+01  1.3247118e+01 -6.0625916e+00  8.7700859e-02
  -2.6373744e-02  4.0147319e-02 -1.0856626e-03  1.0000000e+01]]
0
answer theta :  [-30.035671  15.843011 -17.077667 -22.321722 -18.635263  -9.704083]
pred theta :  [[-179.61038     50.05577    -75.18576     -0.2814718  -79.20096
   -27.872246 ]]

[[ 9.95385230e-01 -4.53882711e-03  7.09917489e-03  3.08857994e+01]
 [-4.71317256e-03  9.81393516e-01 -1.92724308e-03  1.32471180e+01]
 [ 6.98464271e-03 -2.30809720e-03  9.83225942e-01 -6.06259155e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] input_pos 

[[ 0.05691143 -0.4553204  -0.88850685 31.25820289]
 [-0.86797009 -0.46232925  0.18132727 12.982376  ]
 [-0.49334471  0.76087778 -0.42151631 -5.59520353]
 [ 0.          0.          0.          1.        ]] reconstruct_pos
RMSE : 0.706332277623665
위치 차이 :  0.6536239266576845


In [201]:
trained_model.predict(x_tst[:500])


array([[ 5.693722 ,  4.2782044, 12.55663  ,  7.5989614, 17.068867 ,
         6.831456 ],
       [ 6.3803144, 17.021713 , 16.868666 ,  9.067949 , 14.672355 ,
         8.436499 ],
       [10.790721 ,  2.7425747, 14.05033  , 19.072336 , 15.060438 ,
         5.9932327],
       ...,
       [ 6.0922236, 12.668366 ,  4.219946 ,  3.7716146,  6.144619 ,
         9.104296 ],
       [ 3.9970438,  8.273874 ,  4.1142397,  5.4117546,  7.9626074,
        15.274482 ],
       [10.563521 ,  2.9968817, 14.675466 , 16.576593 ,  4.8025517,
         3.5658474]], dtype=float32)

In [20]:
def fk_above_1cm_dist(pred,ans_):
    pred *= 10
    ans = ans_ * 10

    pred = tf.convert_to_tensor(pred)
    ans = tf.cast(ans, pred.dtype)
    print(pred[10])
    print(ans[10])
    pred = tf.experimental.numpy.deg2rad(pred)
    ans = tf.experimental.numpy.deg2rad(ans)
                                        
    
    pred_fk = robot.HTrans_tf(pred)
    ans_fk = robot.HTrans_tf(ans)
    
    print(pred_fk[10])
    print(ans_fk[10])
    dist = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(ans_fk,pred_fk)),axis=1))[:,3]
    #print(dist)
    above_1cm = tf.reduce_sum(tf.cast(dist>1., tf.float32))
    return above_1cm/(robot.batch_size)*100

def monitor(pos_,ans):
    pred = trained_model.predict(pos_)
    
    percent = fk_above_1cm_dist(pred,ans)
    print(percent)
    

    

In [44]:
n = 300
monitor(x_tst[n:n+500],y_tst[n:n+500])

tf.Tensor([ 71.42384 107.4031  142.48212 101.48414 118.65286 -70.54606], shape=(6,), dtype=float32)
tf.Tensor([ 70.574104 104.114655 146.09344  106.730774 110.915825 -60.27131 ], shape=(6,), dtype=float32)
tf.Tensor(
[[ 0.18166347  0.6578538  -0.7309083   8.913367  ]
 [-0.37689355 -0.6399374  -0.6696504   3.5820327 ]
 [-0.90826744  0.3971256   0.1316876   4.6055408 ]
 [ 0.          0.          0.          1.        ]], shape=(4, 4), dtype=float32)
tf.Tensor(
[[ 0.36262956  0.6708339  -0.64690167  9.52347   ]
 [-0.3645089  -0.53675264 -0.7609402   2.0148683 ]
 [-0.8576906   0.5117408   0.04988253  3.5123606 ]
 [ 0.          0.          0.          1.        ]], shape=(4, 4), dtype=float32)
tf.Tensor(77.200005, shape=(), dtype=float32)


In [23]:
y_tst

array([[-1.9748006 , -0.1536458 ,  0.15897976,  1.8154018 ,  1.9899802 ,
        -0.44361714],
       [-1.9748006 , -0.00526308, -0.15897976,  1.9849786 ,  1.9899802 ,
        -0.44361714],
       [-1.9748006 , -1.0477848 ,  1.7194332 , -1.9925052 , -1.9899802 ,
         2.6979756 ],
       ...,
       [-0.4838836 ,  3.1087294 , -0.5919652 ,  1.0979123 ,  2.7987373 ,
        -1.2913666 ],
       [-0.4838836 ,  2.522378  ,  1.4720308 ,  2.7618604 , -2.7987373 ,
         1.850226  ],
       [-0.4838836 , -2.409134  , -1.4720308 , -1.9289368 , -2.7987373 ,
         1.850226  ]], dtype=float32)

In [33]:
x_tst[100]

array([-34.858067  ,  15.09689   ,   7.1604176 ,  -4.5333033 ,
         1.0415554 ,   8.830617  ,  -0.62010443,  40.        ],
      dtype=float32)

In [30]:
robot.position_data[:,3:7] *=10